# v11.3

whereas the last version used matryoshka models, i think it makes more sense to go all-in on the self-similarity thing and simplify the whole process by using the same decoder for all levels

todo's:
- [x] copy & paste stuff that doesn't need to be changed
- [x] copy, paste, & remove references to matryoshka embeddings for modules w/ em
- [x] bring over todo items that haven't been completed from v11.2 and from notes app
- [ ] setup single attention mechanism module for both self & cross? i feel like separate is prolly still easier. and this is kinda just unnecessary extra work
- [x] setup single decoder layer body
- [x] setup concept loss to use MSE and COS simultaneously
- [ ] get predictive attention mask working in crossMQA
- [x] add different types of pooling options to embedding combiner
- [ ] pull concept loss out of model and into training loop so that i can progressively introduce the concept embedding prediction problem rather than deal with moving goalposts from the get-go
- [x] make option for linear layer vs MLP in output from final concept residual states to actual concepts
- [ ] write an inference algorithm that sacrifices quality for efficiency & see how well it works
- [ ] ~~add multiple <|bos|> tokens to put at the beginning of each sequence, one for each level. So that way the model can know what level it's working with and have somewhere to sink attention.~~ the whole "language is fractal" thing means that the model shouldn't need to know. also if an MLP or linear is used for combining concepts then this would be happening implicitly anyways. better to just let a future better tokenizer do its job. 
- [x] make get_batch output values that'd result in actually using the padding token so that it gets trained
- [ ] ~~switch to GaLore optimizer to save ram~~
- [x] implement basic stuff i'm missing like cosine learning rate scheduling (gradient clipping?)
- [ ] move everything into .py files without the LoggingModule parent
- [ ] create big hyperparameter testing script to send to ben
- [ ] would it make sense to have the RoPE theta adjust for the different sequence levels? like divide it by config.combine_factor? i really need to learn more about how RoPE works at an intuitive level rather than just "uuuuuh trig rotations"
- [ ] ~~maybe add learned positional encodings to pooling?~~ nah ur doing too much
- [ ] get the current version running & training so that i can replace TinyShakespeare with TinyStories and make corresponding edits to the dataloader
- [x] make concepts defined recursively
- [ ] setup inference concept selection
      - [ ] add option to select based on cos, mae, and/or mse

further ideas likely for v11.4:
- [ ] setup effective_seq_len_mult and seq_len_list and a bunch of downstream stuff like weird batching at lower sub-levels to allow for hella long effective context lengths. i think what i do here is setup batches such that for a given context length T the concept level above will see nT before and after
- [ ] figure out a way to make config.combine_factor dynamic instead of predetermined. I might be able to do something similar to [Quiet-STAR](https://arxiv.org/abs/2403.09629) to get this to work. Like allow the model to generate multiple outputs and if those outputs help in the actual concept vector generation then keep them in the sequence

# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [2]:
# Importing pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F

# imports for the debugging/demonstration setup
import functools
import inspect

# imports for the tokenizer
from tiny_stories_tokenizer import *

# Imports used for the config
from dataclasses import dataclass
from typing import Optional

# Imports used for the model
import re
from typing import Any, List, Sequence, Tuple, Union

# used for training
import random
import time

# used to save & load models
import json
from dataclasses import asdict

# Demonstration/Debugging wrapper & Module

In [3]:
# this function will be used throughout for debugging/demonstration purposes
# using this is way cleaner than cluttering up our code with print statements
def log_io(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        # Check if logging is enabled globally and for the specific function
        if not self.logging_enabled or func.__name__ in self.disabled_logging_functions:
            return func(self, *args, **kwargs)
        #if not self.logging_enabled:
            #return func(self, *args, **kwargs)

        def log_item(item, name, level=0, is_root=False):
            indent = "    " * level
            if isinstance(item, torch.Tensor):
                print(f"{indent}Tensor '{name}' shape: {item.shape}")
            elif isinstance(item, tuple):
                if is_root and level == 0:
                    # Root level tuple, don't print it as a tuple unless it's a "true" tuple
                    for idx, sub_item in enumerate(item):
                        log_item(sub_item, f"{name}[{idx}]", level)
                else:
                    print(f"{indent}Tuple '{name}':")
                    for idx, sub_item in enumerate(item):
                        log_item(sub_item, f"{name}[{idx}]", level + 1)
            elif isinstance(item, int):
                print(f"{indent}Integer '{name}': Value={item}")
            elif isinstance(item, float):
                print(f"{indent}Float '{name}': Value={item}")
            else:
                print(f"{indent}Other-type '{name}': Type={type(item).__name__}, Value={item}")

        print(f"\n{'='*10}Entering {self.__class__.__name__}.{func.__name__}{'='*10}")
        print("Inputs:")
        arg_names = inspect.getfullargspec(func).args[1:]  # Excluding 'self'
        arg_values = args + tuple(kwargs.values())
        for name, value in zip(arg_names, arg_values):
            log_item(value, name)

        result = func(self, *args, **kwargs)
        print("\nOutputs:")
        if isinstance(result, tuple):
            log_item(result, "output", is_root=True)
        else:
            log_item(result, "output")

        print(f"{'='*10}Exiting {self.__class__.__name__}.{func.__name__}{'='*10}")
        return result
    return wrapper

class LoggingModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.logging_enabled = False
        self.disabled_logging_functions = set()

    def enable_logging(self):
        self.logging_enabled = True

    def disable_logging(self):
        self.logging_enabled = False

    def disable_function_logging(self, func_name):
        self.disabled_logging_functions.add(func_name)

    def enable_function_logging(self, func_name):
        self.disabled_logging_functions.discard(func_name)

# Config & tokenizer

In [4]:
# load the dataset
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# we'll replace this with the TinyStories dataset and the corresponding funcitons below once we've got a regular running model again

# and the tokenizer
tokenizer = get_tokenizer(512)

In [5]:
@dataclass # a class meant specifically to just hold data
class Config:
    """ 
    The default configuration & hyperparameters for my next-concept predictor
    """
    ### boring hyperparameters
    vocab_size: int = tokenizer.vocab_len
    max_seq_len: int = 128
    embed_dim: int = 128
    num_layers: int = 8
    sa_q_heads: int = 4
    sa_kv_heads: int = 1
    #attn_bias: bool = False
    sa_head_dim: int = 32
    mlp_multiplier: int = 2
    theta: float = 100.0
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dropout_rate: float = 0.02
    eps = 1e-6
    norm_affine: bool = True # whether norms should have a linear & bias after them. likely less necessary if you're using RMSNorm
    norm_type = "RMSNorm"  # Options are RMSNorm, CosineNorm and LayerNorm. defaults to RMSNorm
    learning_rate: float = 1e-1
    learning_rate_end: float = 1e-4
    weight_decay: float = 0.01
    max_iters: int = 1000

    ### Concept embedding vectors
    levels: int = 3
    combine_factor: int = 4 # how many lower-level tokens/concepts to combine into the next level's concept
    combine_type: str = 'reshape->mlp_post_reshape->norm' # options to combine are 'sum', 'mean', 'max', 'linear', 'mlp', 'reshape', 
                                                        # 'linear_post_reshape', and 'mlp_post_reshape'
    @property
    def seq_len_list(self):
        return [(self.max_seq_len // (self.combine_factor ** (i-1))) for i in range(1, self.levels + 1)]

    ### Dualcoder cross-attention
    ca_q_heads: int = sa_q_heads
    ca_kv_heads: int = sa_kv_heads
    ca_head_dim: int = sa_head_dim
    ca_use_RoPE: bool = False # True: expands out k & v tensors to be usable with rope. False: leaves k & v same size but no positional encodings
    predictive_mask: bool = False # True: upper-triangular predictive mask to focus model's attention (not currently working). False: no mask like a regular encoder
    predictive_mask_noise: float = None # float: sd of noise to add to predictively masked concept vecs. None: don't implement noise

    ### concept output
    output_layer = 'mlp' # options are 'linear' and 'mlp' which uses the default mlp_multiplier
    # how much to discount each higher level in the loss function compared to the last
    level_loss_weight: float = 1.0 
    # multiple losses can act on the concept vectors at once
    cos_loss: bool = True
    mse_loss: bool = True
    mae_loss: bool = True
    
    ### assertions
    assert sa_q_heads % sa_kv_heads == 0, 'the number of query heads must be divisible by the number of key-value heads in self-attention'
    assert ca_q_heads % ca_kv_heads == 0, 'the number of query heads must be divisible by the number of key-value heads in cross-attention'     
        
config = Config()
print(config)
print(f"sequence length of each model: {config.seq_len_list}")
print(f"loss discounts starting from lowest level: {[config.level_loss_weight**i for i in range(config.levels)]}")

Config(vocab_size=512, max_seq_len=128, embed_dim=128, num_layers=8, sa_q_heads=4, sa_kv_heads=1, sa_head_dim=32, mlp_multiplier=2, theta=100.0, dropout_rate=0.02, norm_affine=True, learning_rate=0.1, learning_rate_end=0.0001, weight_decay=0.01, max_iters=1000, levels=3, combine_factor=4, combine_type='reshape->mlp_post_reshape->norm', ca_q_heads=4, ca_kv_heads=1, ca_head_dim=32, ca_use_RoPE=False, predictive_mask=False, predictive_mask_noise=None, level_loss_weight=1.0, cos_loss=True, mse_loss=True, mae_loss=True)
sequence length of each model: [128, 32, 8]
loss discounts starting from lowest level: [1.0, 1.0, 1.0]


# RoPE

In [6]:
def RoPE(x: torch.Tensor, dim: int, theta: float = 10000.0) -> torch.Tensor:
    """Applies the rotary embedding to the inputted query or key tensor"""
    # Validate that dim is even since we split it by 2 for real and imaginary parts
    if dim % 2 != 0: raise ValueError("Dimension 'dim' must be an even number.")
            
    # Get sequence length
    seq_len = x.size(1)
    device = x.device

    # Dynamically compute frequency cis based on the input sequence length
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2, device=device).float() / dim))
    t = torch.arange(seq_len, device=device)
    freqs = torch.outer(t, freqs).float()
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    # it's important to train on a wide variety of sequence lengths within your context length so that the model learns to generalize

    # Apply rotary embeddings to the input tensor
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    x_out = x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

    return x_out

## maybe precompute frequencies later

i like the idea of pre-computing RoPE embeddings but at this point i don't think it's worth the effort bc i'd have to not only use this code but also pipe the two instantiations of this class through from `Body` all the way to `selfMQA` and `crossMQA` and I'm not even sure if it matters. I really should learn more about RoPE

```Python
# this class has not been implemented nor even tested. on my todo list
class RoPE(nn.Module):
    def __init__(self, 
                 dim: int, 
                 max_seq_len:int = config.max_seq_len, 
                 device: str = config.device):
        super().__init__()
        # Validate that dim is even since we split it by 2 for real and imaginary parts
        if dim % 2 != 0: raise ValueError("Dimension 'dim' must be an even number.")
            
        # Precompute frequencies based on configuration
        theta = config.theta if hasattr(config, 'theta') else 10000.0
        
        freqs = 1.0 / (theta ** (torch.arange(0, config.dim, 2, device=config.device).float() / config.dim))
        t = torch.arange(config.max_seq_len, device=config.device)
        freqs = torch.outer(t, freqs).to(config.device).float()
        
        # Register as buffer to prevent gradient tracking
        self.register_buffer('freqs_cis', torch.polar(torch.ones_like(freqs), freqs)) # complex64

    def forward(self, x):
        # Apply rotary embeddings to the input tensor
        x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
        x_out = torch.view_as_real(x_ * self.freqs_cis.unsqueeze(0)).type_as(x)
        x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
        x_out = x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)
        return x_out
```

# selfMQA

In [7]:
class selfMQA(LoggingModule): # notice thoughout we'll be inheriting from LoggingModule instead of nn.Module
    def __init__(self, config: Config):
        super().__init__()

        self.sa_q_heads = config.sa_q_heads
        self.sa_kv_heads = config.sa_kv_heads
        assert self.sa_q_heads % self.sa_kv_heads == 0
        self.num_queries_per_kv = self.sa_q_heads // self.sa_kv_heads

        self.embed_dim = config.embed_dim
        self.sa_head_dim = config.sa_head_dim
        self.theta = config.theta
        self.dropout_rate = config.dropout_rate

        #self.Wqkv = nn.Linear(self.embed_dim, 
        #                      (self.sa_q_heads + 2 * self.sq_kv_heads) * self.sa_head_dim, 
        #                      bias = config.attn_bias)
        #self.Wo = nn.Linear(self.sa_q_heads * self.sa_head_dim,
        #                    self.embed_dim,
        #                    bias = config.attn_bias)
        
        self.Wqkv = nn.Parameter(torch.Tensor(self.embed_dim, (self.sa_q_heads + 2 * self.sa_kv_heads) * self.sa_head_dim))
        nn.init.uniform_(self.Wqkv, -((1 / self.embed_dim) ** 0.5), (1 / self.embed_dim) ** 0.5)

        self.Wo = nn.Parameter(torch.Tensor(self.sa_q_heads * self.sa_head_dim, self.embed_dim))
        nn.init.uniform_(self.Wo, -((1 / (self.sa_q_heads * self.sa_head_dim)) ** 0.5), (1 / (self.sa_q_heads * self.sa_head_dim)) ** 0.5)

        # for our attention mask we'll create a boolean mask that'll later be turned into large negative values
        self.mask = torch.tril(torch.ones((config.max_seq_len, config.max_seq_len), dtype=torch.uint8)
                              ).view(1, 1, config.max_seq_len, config.max_seq_len).to(dtype=torch.bool)

    @log_io
    def forward(self, x: torch.Tensor, training: bool = False) -> torch.Tensor:
        # Extracts batch size and input sequence length from the hidden states tensor.
        batch_size, input_len, _ = x.shape

        # splicing our primary projection to get the correct sub-matrices
        Wq, Wk, Wv = self.weight_splicing(self.Wqkv)
        # technically self.weight_splicing has access to self.Wqkv & Wo but this way our debugger can see them

        # Applies the linear projection to the hidden state to retrieve our q, k & v projections
        xq = F.dropout(x @ Wq, p=self.dropout_rate, training=training) # also dropout if we're training
        xk = F.dropout(x @ Wk, p=self.dropout_rate, training=training)
        xv = F.dropout(x @ Wv, p=self.dropout_rate, training=training)

        # Reshapes each of the Q, K, and V tensors to separate the heads and align the dimensions for attention operations.
        xq = xq.view(batch_size, -1, self.sa_q_heads, self.sa_head_dim)
        xk = xk.view(batch_size, -1, self.sa_kv_heads, self.sa_head_dim)
        xv = xv.view(batch_size, -1, self.sa_kv_heads, self.sa_head_dim)

        # Applies rotary positional embeddings to queries and keys to incorporate positional information.
        xq, xk = self.RoPE(xq, xk)

        # If the number of KV heads is different from the number of query heads, adjusts keys and values to match the query heads count.
        if self.sa_kv_heads != self.sa_q_heads:
            xk, xv = self.match_headcount(xk, xv) # [batch_size, input_len, n_local_heads, sa_head_dim]

        # Transposes Q, K, and V tensors to align them for the batch matrix multiplication in attention calculation.
        xq = xq.transpose(1, 2) # [batch_size, n_local_heads, input_len, sa_head_dim]
        xk = xk.transpose(1, 2)
        xv = xv.transpose(1, 2)

        # Calculates attention scores by performing a batch matrix multiplication between queries and keys, followed by scaling.
        logits = self.attend(xq, xk) # [batch_size, n_local_heads, input_len, input_len]
        
        # Applies the lower-triangular mask to the attention logits
        logits = self.apply_mask(logits, input_len)

        # applies values to get final output
        output = self.calc_output(logits, xv, batch_size, input_len, training) 

        # Applies the final linear projection to the attention output, mapping it back to self.embed_dim
        return F.dropout(output @ self.Wo, p=self.dropout_rate, training=training) # also dropout if we're training

    @log_io
    def weight_splicing(self, Wqkv):
        Wq, Wk, Wv = Wqkv.split([self.sa_q_heads * self.sa_head_dim,
                                 self.sa_kv_heads * self.sa_head_dim,
                                 self.sa_kv_heads * self.sa_head_dim],dim = -1)
        return Wq, Wk, Wv

    @log_io
    def RoPE(self, xq, xk):
        xq = RoPE(xq, self.sa_head_dim, self.theta)
        xk = RoPE(xk, self.sa_head_dim, self.theta)
        return xq, xk

    @log_io
    def match_headcount(self, xk, xv):
        xk = torch.repeat_interleave(xk, self.num_queries_per_kv, dim=2)
        xv = torch.repeat_interleave(xv, self.num_queries_per_kv, dim=2)
        return xk, xv

    @log_io
    def attend(self, xq, xk):
        return torch.matmul(xq, xk.transpose(2, 3)) * (self.sa_head_dim ** -0.5)
        
    @log_io
    def apply_mask(self, logits, input_len):
        return torch.where(self.mask[..., :input_len, :input_len].expand_as(logits),
                           logits,
                           torch.tensor(-1e30, device=logits.device, dtype=logits.dtype))
    
    @log_io
    def calc_output(self, logits, xv, batch_size, input_len, training):
        # Applies softmax to the logits to obtain attention probabilities
        scores = F.softmax(logits, dim=-1)

        # also applies dropout if we're training
        scores = F.dropout(scores, p=self.dropout_rate, training=training)
        
        # Computes the weighted sum of values based on the attention scores to obtain the output of the attention mechanism.
        output = scores @ xv # [batch_size, n_local_heads, input_len, sa_head_dim]

        # Reshapes the attention output to match the expected output dimensions, combining the heads back into the hidden dimension.
        return output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)

## demonstration/debugging
I've setup these little snippets after each nn.Module to help you see what's happening and for my own debugging

In [8]:
# Create an instance of selfMQA
module = selfMQA(config)

# Initially, logging is disabled
# Enable logging
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('weight_splicing')
#module.disable_function_logging('RoPE')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('apply_mask')
#module.disable_function_logging('calc_output')

# Call the forward method - logging will occur
output = module(torch.randn(32,config.max_seq_len,config.embed_dim))#, training=True)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, output


==========Entering selfMQA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])

==========Entering selfMQA.weight_splicing==========
Inputs:
Tensor 'Wqkv' shape: torch.Size([256, 256])

Outputs:
Tensor 'output[0]' shape: torch.Size([256, 128])
Tensor 'output[1]' shape: torch.Size([256, 64])
Tensor 'output[2]' shape: torch.Size([256, 64])
==========Exiting selfMQA.weight_splicing==========

==========Entering selfMQA.RoPE==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 256, 2, 64])
Tensor 'xk' shape: torch.Size([32, 256, 1, 64])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 256, 2, 64])
Tensor 'output[1]' shape: torch.Size([32, 256, 1, 64])
==========Exiting selfMQA.RoPE==========

==========Entering selfMQA.match_headcount==========
Inputs:
Tensor 'xk' shape: torch.Size([32, 256, 1, 64])
Tensor 'xv' shape: torch.Size([32, 256, 1, 64])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 256, 2, 64])
Tensor 'output[1]' shape: torch.Size([32, 256, 2, 64])
==

# MLP

In [8]:
class MLP(LoggingModule):
    def __init__(self,
                 embed_dim: int,
                 mlp_multiplier: int,
                 output_dim: int,
                 dropout_rate: float = 0.1):
        super().__init__()
        self.mlp_multiplier = mlp_multiplier
        self.hidden_size = embed_dim * mlp_multiplier
        self.dropout_rate = dropout_rate

        # the gate, up and down projections
        self.gate_proj = nn.Linear(embed_dim, self.hidden_size)
        self.up_proj = nn.Linear(embed_dim, self.hidden_size)
        self.down_proj = nn.Linear(self.hidden_size, output_dim)
        
    @log_io
    def forward(self, x: torch.Tensor, training: bool = False ) -> torch.Tensor:
        output = self.down_proj(F.gelu(self.gate_proj(x)) * self.up_proj(x))
        return F.dropout(output, p=self.dropout_rate, training=training)

## demonstration/debugging

In [10]:
module = MLP(config.embed_dim, config.mlp_multiplier, config.embed_dim, config.dropout_rate)
module.enable_logging()
output = module(torch.randn(32,config.max_seq_len,config.embed_dim))
del module, output


==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting MLP.forward==========


# Norms

In [9]:
class Norm(LoggingModule):
    def __init__(self, config):
        super().__init__()
        self.eps = config.eps
        self.affine = config.norm_affine
        self.dropout_rate = config.dropout_rate
        self.type = config.norm_type

        # Initialize weight and bias parameters for affine transformation
        # We start with ones for weight to keep the original scale initially, and zeros for bias.
        self.w = nn.Parameter(torch.ones(config.embed_dim))
        self.b = nn.Parameter(torch.zeros(config.embed_dim))

    @log_io
    def forward(self, x: torch.Tensor, training: bool = False) -> torch.Tensor:
        # Normalize the input tensor
        if self.type == "CosineNorm":
            x = self.CosineNorm(x)
        elif self.type == "LayerNorm":
            x = self.LayerNorm(x)
        else: # defaults to RMSNorm bc that's the most commonly used nowadays
            x = self.RMSNorm(x)

        if self.affine: # Optionally apply the affine transformation and dropout if we're training
            x = F.dropout(x * self.w + self.b, p=self.dropout_rate, training=training)
            
        return x

    @log_io
    def CosineNorm(self, x):
        # normalize x by dividing by its L2 norm along the last dimension.
        # this places x on the unit hypersphere centered at the origin
        # Add a small constant to the denominator to avoid division by zero.
        return x / torch.norm(x, p=2, dim=-1, keepdim=True).clamp(min=self.eps)

    @log_io
    def LayerNorm(self, x): # nn.LayerNorm() exists but might as well make it from scratch if we have to do the other two
        # normalize x by subtracting by its mean then dividing by its variance
        # this places x on a hypersphere of radius sqrt(dimension) centered at the origin
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        return (x - mean) / torch.sqrt(var + self.eps)

    @log_io
    def RMSNorm(self, x):
        # normalize x by dividing by its root-mean-square along the last dimension
        # this places x on a hypersphere of radius sqrt(dimension) with no certain center
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

## demonstration/debugging

In [12]:
module = Norm(config)
module.enable_logging()

### disabling printing for sub-functions
#module.disable_function_logging('RMSNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('CosineNorm')

output = module(torch.randn(32, config.max_seq_len, config.embed_dim))
del module, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])

==========Entering Norm.RMSNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Norm.RMSNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Norm.forward==========


# crossMQA

In [10]:
class crossMQA(LoggingModule):
    def __init__(self, config: Config):
        super().__init__()

        self.ca_q_heads = config.ca_q_heads
        self.ca_kv_heads = config.ca_kv_heads
        assert self.ca_q_heads % self.ca_kv_heads == 0
        self.num_queries_per_kv = self.ca_q_heads // self.ca_kv_heads
        self.embed_dim = config.embed_dim
        self.ca_head_dim = config.ca_head_dim
        self.sa_head_dim = config.sa_head_dim # used only for an assertion to make sure sizes will fit
        self.combine_factor = config.combine_factor # used only for an assertion to make sure sizes will fit
        self.theta = config.theta
        self.use_RoPE = config.ca_use_RoPE
        self.dropout_rate = config.dropout_rate
        self.predictive_mask = config.predictive_mask

        self.Wqkv = nn.Parameter(torch.Tensor(self.embed_dim, (self.ca_q_heads + 2 * self.ca_kv_heads) * self.ca_head_dim))
        nn.init.uniform_(self.Wqkv, -((1 / self.embed_dim) ** 0.5), (1 / self.embed_dim) ** 0.5)
        
        self.Wo = nn.Parameter(torch.Tensor(self.ca_q_heads * self.ca_head_dim, self.embed_dim))
        nn.init.uniform_(self.Wo, -((1 / (self.ca_q_heads * self.ca_head_dim)) ** 0.5), (1 / (self.ca_q_heads * self.ca_head_dim)) ** 0.5)

    @log_io
    def forward(self, 
                x: torch.Tensor, # the current level tensor, sometimes a resid state full of tokens & sometimes concepts
                c: torch.Tensor, # the upper level tensor, always a resid state full of concept vecs
                training: bool = False
               ) -> torch.Tensor:
        
        # Extracts batch size and input sequence length from the hidden states tensor.
        batch_size, input_len_x, _ = x.shape
        batch_size_c, input_len_c, _ = c.shape
        assert batch_size == batch_size_c

        # splicing our projection to get the correct sub-matrices
        Wq, Wk, Wv = self.weight_splicing(self.Wqkv)

        # Applies the linear projection to the hidden state to retrieve our q, k & v projections
        xq = F.dropout(x @ Wq, p=self.dropout_rate, training=training) # also applies dropout if we're training
        ck = F.dropout(c @ Wk, p=self.dropout_rate, training=training)
        cv = F.dropout(c @ Wv, p=self.dropout_rate, training=training)

        # Reshapes each of the Q, K, and V tensors to separate the heads and align the dimensions for attention operations.
        xq = xq.view(batch_size, -1, self.ca_q_heads, self.ca_head_dim)
        ck = ck.view(batch_size, -1, self.ca_kv_heads, self.ca_head_dim)
        cv = cv.view(batch_size, -1, self.ca_kv_heads, self.ca_head_dim)

        # IF we want to use RoPE (doesn't fully make sense to i don't think)
        if self.use_RoPE:
            expand = input_len_x // input_len_c
            ck = ck.repeat_interleave(expand, dim=1) 
            cv = cv.repeat_interleave(expand, dim=1) # values need to be expanded for their use later on if we do this

            # Applies rotary positional embeddings to queries and keys to incorporate positional information.
            xq, ck = self.RoPE(xq, ck)

        # If the number of KV heads is different from the number of query heads, adjusts keys and values to match the query heads count.
        if self.ca_kv_heads != self.ca_q_heads:
            ck, cv = self.match_headcount(ck, cv) # [batch_size, input_len, n_local_heads, sa_head_dim]

        # Transposes Q, K, and V tensors to align them for the batch matrix multiplication in attention calculation.
        xq = xq.transpose(1, 2) # [batch_size, n_local_heads, input_len, ca_head_dim]
        ck = ck.transpose(1, 2)
        cv = cv.transpose(1, 2)

        # Calculates attention scores by performing a batch matrix multiplication between queries and keys, followed by scaling.
        logits = self.attend(xq, ck) # [batch_size, n_local_heads, input_len, input_len]
        
        # Optionally applies the upper-triangular mask to the attention logits
        if self.predictive_mask:
            logits = self.apply_mask(logits, input_len_x, input_len_c)

        # applies values to get final output
        output = self.calc_output(logits, cv, batch_size, input_len_x, training)

        # Applies the final linear projection to the attention output, mapping it back to d_i. 
        return F.dropout(output @ self.Wo, p=self.dropout_rate, training=training) # and dropout if we're training

    @log_io
    def weight_splicing(self, Wqkv):
        Wq, Wk, Wv = Wqkv.split([self.ca_q_heads * self.ca_head_dim,
                                 self.ca_kv_heads * self.ca_head_dim,
                                 self.ca_kv_heads * self.ca_head_dim],dim = -1)
        return Wq, Wk, Wv
        
    @log_io
    def RoPE(self, xq, xk):
        xq = RoPE(xq, self.ca_head_dim, self.theta)
        xk = RoPE(xk, self.ca_head_dim, self.theta)
        return xq, xk

    @log_io
    def match_headcount(self, xk, xv):
        xk = torch.repeat_interleave(xk, self.num_queries_per_kv, dim=2)
        xv = torch.repeat_interleave(xv, self.num_queries_per_kv, dim=2)
        return xk, xv

    @log_io
    def attend(self, xq, ck):
        return torch.matmul(xq, ck.transpose(2, 3)) * (self.ca_head_dim ** -0.5)
        
    @log_io
    def apply_mask(self, logits, input_len_x, input_len_c):
        self.mask = torch.triu(torch.ones((config.max_seq_len, input_len_c), dtype=torch.uint8)
                                  ).view(1, 1, config.max_seq_len, input_len_c).to(dtype=torch.bool)
        
        return torch.where(self.mask[..., :input_len_x, :input_len_c].expand_as(logits),
                           logits,
                           torch.tensor(-1e30, device=logits.device, dtype=logits.dtype))
    
    @log_io
    def calc_output(self, logits, cv, batch_size, input_len_x, training):
        # Applies softmax to the logits to obtain attention probabilities
        scores = F.softmax(logits, dim=-1)

        # also applies dropout if we're training
        scores = F.dropout(scores, p=self.dropout_rate, training=training)
        
        # Computes the weighted sum of values based on the attention scores to obtain the output of the attention mechanism.
        output = scores @ cv # [batch_size, n_local_heads, input_len, sa_head_dim]

        # Reshapes the attention output to match the expected output dimensions, combining the heads back into the hidden dimension.
        return output.transpose(1, 2).contiguous().view(batch_size, input_len_x, -1)

## demonstration/debugging

at one point this one had been giving me trouble so here's multiple different config.levels setups for ya

In [14]:
hold1 = config.levels
config.levels = 2
hold2 = config.predictive_mask
config.predictive_mask = False
module = crossMQA(config)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('weight_splicing')
#module.disable_function_logging('RoPE')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('apply_mask')
#module.disable_function_logging('calc_output')

x0 = torch.randn(32, config.max_seq_len, config.embed_dim)
c1 = torch.randn(32, config.max_seq_len // config.combine_factor, config.embed_dim)
output = module(x0, c1)
config.levels = hold1
config.predictive_mask = hold2
del hold1, hold2, module, x0, c1, output


==========Entering crossMQA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Tensor 'c' shape: torch.Size([32, 64, 256])

==========Entering crossMQA.weight_splicing==========
Inputs:
Tensor 'Wqkv' shape: torch.Size([256, 256])

Outputs:
Tensor 'output[0]' shape: torch.Size([256, 128])
Tensor 'output[1]' shape: torch.Size([256, 64])
Tensor 'output[2]' shape: torch.Size([256, 64])
==========Exiting crossMQA.weight_splicing==========

==========Entering crossMQA.match_headcount==========
Inputs:
Tensor 'xk' shape: torch.Size([32, 64, 1, 64])
Tensor 'xv' shape: torch.Size([32, 64, 1, 64])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 64, 2, 64])
Tensor 'output[1]' shape: torch.Size([32, 64, 2, 64])
==========Exiting crossMQA.match_headcount==========

==========Entering crossMQA.attend==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 2, 256, 64])
Tensor 'ck' shape: torch.Size([32, 2, 64, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 2, 256, 64])
==

In [15]:
hold1 = config.levels
config.levels = 3
hold2 = config.predictive_mask
config.predictive_mask = False
module = crossMQA(config)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('weight_splicing')
#module.disable_function_logging('RoPE')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('apply_mask')
#module.disable_function_logging('calc_output')

c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
output = module(c1, c2)
config.levels = hold1
config.predictive_mask = hold2
del hold1, hold2, module, c1, c2, output


==========Entering crossMQA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 256])
Tensor 'c' shape: torch.Size([32, 16, 256])

==========Entering crossMQA.weight_splicing==========
Inputs:
Tensor 'Wqkv' shape: torch.Size([256, 256])

Outputs:
Tensor 'output[0]' shape: torch.Size([256, 128])
Tensor 'output[1]' shape: torch.Size([256, 64])
Tensor 'output[2]' shape: torch.Size([256, 64])
==========Exiting crossMQA.weight_splicing==========

==========Entering crossMQA.match_headcount==========
Inputs:
Tensor 'xk' shape: torch.Size([32, 16, 1, 64])
Tensor 'xv' shape: torch.Size([32, 16, 1, 64])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 16, 2, 64])
Tensor 'output[1]' shape: torch.Size([32, 16, 2, 64])
==========Exiting crossMQA.match_headcount==========

==========Entering crossMQA.attend==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 2, 64, 64])
Tensor 'ck' shape: torch.Size([32, 2, 16, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 2, 64, 16])
=====

## working on crossMQA predictive mask

In [40]:
input_len = 10
combine_factor = 4
h = 1
head_dim = 8
b = 1

offset = combine_factor - (input_len % combine_factor)
x = torch.randn(b, input_len + offset, h, head_dim)
print(f'x: {x.shape}\n{x}')

c = torch.randn(b, (input_len + combine_factor) // combine_factor, h, head_dim)
print(f'c: {c.shape}\n{c}')

x = x.transpose(1, 2)
c = c.transpose(1, 2)

a = x @ c.transpose(2,3)
print(f'a: {a.shape}\n{a}')

mask_init = torch.triu(torch.ones(b,h,a.shape[-1], a.shape[-1]), diagonal=1)
print(f'mask_init: {mask_init.shape}\n{mask_init}')

mask = mask_init.repeat_interleave(combine_factor, dim=2)
print(f'mask: {mask.shape}\n{mask}')

x: torch.Size([1, 12, 1, 8])
tensor([[[[-1.6085, -0.4912, -1.0831,  2.0253, -1.0600,  2.1828,  1.5007,
            0.4344]],

         [[-0.5917, -0.3771,  1.2626, -0.3879,  0.5883, -1.0096, -1.0503,
           -0.8059]],

         [[-2.3746,  0.0553,  0.0055, -0.4405, -0.5536, -1.0956, -0.6217,
           -0.2537]],

         [[-0.5987, -1.1044,  0.5889, -0.6850, -0.3425,  0.7003, -0.8719,
            0.9681]],

         [[-0.9842, -0.7047,  0.9958,  0.0113, -1.5084,  1.8332,  0.8966,
            0.3623]],

         [[-1.6388, -0.8443,  0.3519,  0.5995,  1.6932, -0.1226, -1.1139,
            1.0743]],

         [[ 0.2063, -1.2685,  1.5661, -0.6124, -0.9883,  0.7585, -0.0369,
           -0.8424]],

         [[-0.0581,  0.7860, -0.7158,  0.5363, -0.9425, -0.3531,  0.2469,
           -0.5569]],

         [[ 1.1262,  0.8942, -0.0777,  0.4163,  2.2315, -1.0251, -0.1367,
           -0.9200]],

         [[ 1.3204,  1.4768, -0.8936, -0.6451, -0.1610, -0.1215, -0.4567,
            0.1247]],

 

In [58]:
hold1 = config.levels
config.levels = 2
hold2 = config.predictive_mask
config.predictive_mask = True
module = crossMQA(config)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('weight_splicing')
#module.disable_function_logging('RoPE')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('apply_mask')
#module.disable_function_logging('calc_output')

x0 = torch.randn(32, config.max_seq_len, config.embed_dim)
c1 = torch.randn(32, config.max_seq_len // config.combine_factor, config.embed_dim)
output = module(x0, c1)
config.levels = hold1
config.predictive_mask = hold2
del hold1, hold2, module, x0, c1, output


==========Entering crossMQA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Tensor 'c' shape: torch.Size([32, 64, 256])

==========Entering crossMQA.weight_splicing==========
Inputs:
Tensor 'Wqkv' shape: torch.Size([256, 256])

Outputs:
Tensor 'output[0]' shape: torch.Size([256, 128])
Tensor 'output[1]' shape: torch.Size([256, 64])
Tensor 'output[2]' shape: torch.Size([256, 64])
==========Exiting crossMQA.weight_splicing==========

==========Entering crossMQA.match_headcount==========
Inputs:
Tensor 'xn' shape: torch.Size([32, 64, 1, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 2, 64])
==========Exiting crossMQA.match_headcount==========

==========Entering crossMQA.match_headcount==========
Inputs:
Tensor 'xn' shape: torch.Size([32, 64, 1, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 2, 64])
==========Exiting crossMQA.match_headcount==========

==========Entering crossMQA.attend==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 2, 2

RuntimeError: The expanded size of the tensor (64) must match the existing size (256) at non-singleton dimension 3.  Target sizes: [32, 2, 256, 64].  Tensor sizes: [1, 1, 256, 256]

# Layer

we implement cross-attention inbetween self-attention and MLP like is done in Attention is All You Need

In [11]:
class Layer(LoggingModule):
    def __init__(self, config: Config):
        super().__init__()

        self.pre_self_mqa_norm = Norm(config)
        self.self_mqa = selfMQA(config)
        self.post_self_mqa_norm = Norm(config)
        
        self.pre_cross_mqa_x_norm = Norm(config)
        self.pre_cross_mqa_c_norm = Norm(config)
        self.cross_mqa = crossMQA(config)
        self.post_cross_mqa_norm = Norm(config)
        
        self.pre_mlp_norm = Norm(config)
        self.mlp = MLP(config.embed_dim, config.mlp_multiplier, config.embed_dim, config.dropout_rate)
        self.post_mlp_norm = Norm(config)

    @log_io
    def forward(self, 
                x: torch.Tensor,
                c: torch.Tensor = None,
                training: bool = False,
               ) -> torch.Tensor:
        x = x + self.self_mqa_connection(x, training)
        if c is not None:
            x = x + self.cross_mqa_connection(x, c, training)
        x = x + self.mlp_connection(x, training)
        return x

    @log_io
    def self_mqa_connection(self, x, training):
        return self.post_self_mqa_norm(self.self_mqa(self.pre_self_mqa_norm(x, training), training), training)

    @log_io
    def cross_mqa_connection(self, x, c, training):
        return self.post_cross_mqa_norm(self.cross_mqa(self.pre_cross_mqa_x_norm(x, training), 
                                                       self.pre_cross_mqa_c_norm(c, training), training), training)
    @log_io
    def mlp_connection(self, x, training):
        return self.post_mlp_norm(self.mlp(self.pre_mlp_norm(x, training), training), training)

## demonstration/debugging

### no predictive mask

In [17]:
module = Layer(config)
module.enable_logging()

### enabling printing for sub-modules
#module.pre_self_mqa_norm.enable_logging()
#module.self_mqa.enable_logging()
#module.post_self_mqa_norm.enable_logging()
#module.pre_cross_mqa_norm.enable_logging()
#module.cross_mqa.enable_logging()
#module.post_cross_mqa_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

### disabling printing for sub-functions
#module.disable_function_logging('self_mqa_connection')
#module.disable_function_logging('cross_mqa_connection')
#module.disable_function_logging('mlp_connection')

output = module(torch.randn(32,config.max_seq_len,config.embed_dim))
del module, output


==========Entering Layer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])

==========Entering Layer.self_mqa_connection==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Integer 'training': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Layer.self_mqa_connection==========

==========Entering Layer.mlp_connection==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Integer 'training': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Layer.mlp_connection==========

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Layer.forward==========


In [18]:
hold = config.predictive_mask
config.predictive_mask = False
module = Layer(config)
module.enable_logging()

### enabling printing for sub-modules
#module.pre_self_mqa_norm.enable_logging()
#module.self_mqa.enable_logging()
#module.post_self_mqa_norm.enable_logging()
#module.pre_cross_mqa_norm.enable_logging()
#module.cross_mqa.enable_logging()
#module.post_cross_mqa_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

### disabling printing for sub-functions
#module.disable_function_logging('self_mqa_connection')
#module.disable_function_logging('cross_mqa_connection')
#module.disable_function_logging('mlp_connection')

x = torch.randn(32, config.max_seq_len, config.embed_dim)
c = torch.randn(32, config.max_seq_len // config.combine_factor, config.embed_dim)
output = module(x, c)
config.predictive_mask = hold
del hold, module, output


==========Entering Layer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Tensor 'c' shape: torch.Size([32, 64, 256])

==========Entering Layer.self_mqa_connection==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Integer 'training': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Layer.self_mqa_connection==========

==========Entering Layer.cross_mqa_connection==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Tensor 'c' shape: torch.Size([32, 64, 256])
Integer 'training': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Layer.cross_mqa_connection==========

==========Entering Layer.mlp_connection==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 256])
Integer 'training': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256])
==========Exiting Layer.mlp_connection==========

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 256]

### prolly need to do more debugging once predictive mask works

# embedding vector combination function

In [12]:
class SumModule(LoggingModule): 
    @log_io
    def forward(self, x):
        return x.sum(dim=2)
class MeanModule(LoggingModule):    
    @log_io
    def forward(self, x):
        return x.mean(dim=2)
class MaxModule(LoggingModule):    
    @log_io
    def forward(self, x):
        return x.max(dim=2).values
class ReshapeModule(LoggingModule):
    @log_io
    def forward(self, x):
        return x.reshape(x.shape[0], x.shape[1], -1)

class CombineEmbeddings(LoggingModule):
    def __init__(self, config: Config, padding_vector: torch.Tensor):
        super().__init__()
        self.embed_dim = config.embed_dim
        self.combine_factor = config.combine_factor
        #self.padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
        self.padding_vector = padding_vector

        # Initialize operation chain
        self.operation_chain = nn.Sequential()
        operation_map = {
            "sum": SumModule(), # cannot go before or after reshape
            "mean": MeanModule(), # cannot go before or after reshape
            "max": MaxModule(), # cannot go before or after reshape
            "norm": Norm(config), # cannot go before or after reshape
            "linear_post_reshape": nn.Linear(config.embed_dim * config.combine_factor, config.embed_dim), # must go after reshape
            "linear": nn.Linear(config.embed_dim, config.embed_dim), # cannot go after reshape
            "mlp_post_reshape": MLP(config.embed_dim * config.combine_factor, config.mlp_multiplier, config.embed_dim, config.dropout_rate), # must go after reshape
            "mlp": MLP(config.embed_dim, config.mlp_multiplier, config.embed_dim, config.dropout_rate), # cannot go after reshape
            "reshape": ReshapeModule()}
        index = 0  # Initialize a counter to create unique module names
        for op in config.combine_type.split("->"):
            if op in operation_map:
                unique_op_name = f"{op}_{index}"  # Append the index to the operation name
                self.operation_chain.add_module(unique_op_name, operation_map[op])
                index += 1  # Increment the index for the next operation
            else:
                raise ValueError(f"Pooling operation {op} is not a valid operation")

    
    @log_io
    def forward(self, tensor, combine_factor):
        # this function will apply itself recursively for higher levels
        assert combine_factor % self.combine_factor == 0, f'combine factor={combine_factor} is not a multiple of config.combine_factor={config.combine_factor}'
        if combine_factor // self.combine_factor != 1:
            tensor = self.forward(tensor, combine_factor // self.combine_factor)
            
        b, t, d = tensor.shape
            
        # Calculate the necessary amount of padding
        remainder = t % self.combine_factor
        padding_needed = 0 if remainder == 0 else self.combine_factor - remainder
        
        if padding_needed > 0:
            # Replicate the padding vector the necessary number of times
            padding = self.padding_vector.repeat(padding_needed, 1).unsqueeze(0).expand(b, -1, -1)
            tensor = torch.cat([padding, tensor], dim=1)
        
        # Update t after padding
        t_padded = t + padding_needed
        
        # Reshape the tensor to group 'combine_factor' entries along the t dimension
        reshaped_tensor = tensor.view(b, t_padded // self.combine_factor, self.combine_factor, d)
        
        # the actual combination operation
        combined_tensor = self.operation_chain(reshaped_tensor)
        assert combined_tensor.shape[0] == b, f"b={b}; pooling operation order is invalid. output shape: {combined_tensor.shape}"
        assert combined_tensor.shape[1] == t_padded // self.combine_factor, f"t={t_padded // self.combine_factor}; pooling operation order is invalid. output shape: {combined_tensor.shape}"
        assert combined_tensor.shape[2] == self.embed_dim, f"d={self.embed_dim}; pooling operation order is invalid. output shape: {combined_tensor.shape}"
        
        return combined_tensor

## demonstration/debugging

### testing different pooling types

In [25]:
hold1 = config.combine_type
hold2 = config.combine_factor
config.combine_type = 'mlp->sum->mlp->norm'
config.combine_factor = 4
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.mlp_0.enable_logging()
module.operation_chain.sum_1.enable_logging()
module.operation_chain.mlp_2.enable_logging()
module.operation_chain.norm_3.enable_logging()

x = torch.randn(32, config.max_seq_len, config.embed_dim)
output = module(x, config.combine_factor)
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 128, 128])
Integer 'combine_factor': Value=4

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 4, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 4, 128])
==========Exiting MLP.forward==========

==========Entering SumModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 4, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
==========Exiting SumModule.forward==========

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
==========Exiting MLP.forward==========

==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 128])

==========Entering Norm.RMSNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
===

In [26]:
hold1 = config.combine_type
hold2 = config.combine_factor
config.combine_type = 'reshape->mlp_post_reshape'
config.combine_factor = 2
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.reshape_0.enable_logging()
module.operation_chain.mlp_post_reshape_1.enable_logging()

x = torch.randn(32, config.max_seq_len, config.embed_dim)
output = module(x, config.combine_factor)
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 128, 128])
Integer 'combine_factor': Value=2

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 256])
==========Exiting ReshapeModule.forward==========

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 256])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting MLP.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting CombineEmbeddings.forward==========


### off-lengths that'll require padding

In [27]:
hold1 = config.combine_type
hold2 = config.combine_factor
config.combine_type = 'reshape->linear_post_reshape'
config.combine_factor = 2
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.reshape_0.enable_logging()
# the linear is here it's just not compatible with our printing and i'm too lazy to build a wrapper for it

x = torch.randn(32, config.max_seq_len-1, config.embed_dim)
output = module(x, config.combine_factor)
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 127, 128])
Integer 'combine_factor': Value=2

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 256])
==========Exiting ReshapeModule.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting CombineEmbeddings.forward==========


In [28]:
hold1 = config.combine_type
hold2 = config.combine_factor
config.combine_type = 'max'
config.combine_factor = 4
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.max_0.enable_logging()

x = torch.randn(32, config.max_seq_len-3, config.embed_dim)
print(x[0,0,:8])
output = module(x, config.combine_factor)
print(output[0,0,:8])
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, x, output
module = None

tensor([ 0.2387, -1.1476,  0.0282, -1.5950, -1.6658,  0.6434, -0.7313,  0.3435])

==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 125, 128])
Integer 'combine_factor': Value=4

==========Entering MaxModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 4, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
==========Exiting MaxModule.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
==========Exiting CombineEmbeddings.forward==========
tensor([0.2387, 0.0000, 0.0282, 0.0000, 0.0000, 0.6434, 0.0000, 0.3435],
       grad_fn=<SliceBackward0>)


In [29]:
hold1 = config.combine_type
hold2 = config.combine_factor
config.combine_type = 'mean'
config.combine_factor = 4
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.mean_0.enable_logging()

x = torch.randn(32, config.max_seq_len-4, config.embed_dim)
output = module(x, config.combine_factor)
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 124, 128])
Integer 'combine_factor': Value=4

==========Entering MeanModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 31, 4, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 31, 128])
==========Exiting MeanModule.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 31, 128])
==========Exiting CombineEmbeddings.forward==========


### it has to calculate recursively for multiple levels whenever reshape is involved

In [39]:
hold1 = config.combine_type
hold2 = config.combine_factor
hold3 = config.levels
config.combine_type = 'mlp->sum->mlp->norm'
config.combine_factor = 4
config.levels = 3
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.mlp_0.enable_logging()
module.operation_chain.sum_1.enable_logging()
module.operation_chain.mlp_2.enable_logging()
module.operation_chain.norm_3.enable_logging()

x = torch.randn(32, config.max_seq_len, config.embed_dim)
output = module(x, config.combine_factor**(config.levels-1))
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, hold3, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 128, 128])
Integer 'combine_factor': Value=16
combine_factor: 16, combine_factor // self.combine_factor: 4

==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 128, 128])
Integer 'combine_factor': Value=4
combine_factor: 4, combine_factor // self.combine_factor: 1

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 4, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 4, 128])
==========Exiting MLP.forward==========

==========Entering SumModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 4, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
==========Exiting SumModule.forward==========

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 128])
==========Exiting MLP.

### recursive and off-length sequence

In [42]:
hold1 = config.combine_type
hold2 = config.combine_factor
hold3 = config.levels
config.combine_type = 'reshape->linear_post_reshape'
config.combine_factor = 2
config.levels = 3
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.reshape_0.enable_logging()

x = torch.randn(32, config.max_seq_len - 1, config.embed_dim)
output = module(x, config.combine_factor**(config.levels-1))
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, hold3, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 127, 128])
Integer 'combine_factor': Value=4
combine_factor: 4, combine_factor // self.combine_factor: 2

==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 127, 128])
Integer 'combine_factor': Value=2
combine_factor: 2, combine_factor // self.combine_factor: 1

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 256])
==========Exiting ReshapeModule.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting CombineEmbeddings.forward==========

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 256])
==========Exiting ReshapeModule.forward==========

Outputs:
Tensor 'output' shape: torch.

In [43]:
hold1 = config.combine_type
hold2 = config.combine_factor
hold3 = config.levels
config.combine_type = 'reshape->linear_post_reshape'
config.combine_factor = 2
config.levels = 3
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.reshape_0.enable_logging()

x = torch.randn(32, config.max_seq_len - 3, config.embed_dim)
output = module(x, config.combine_factor**(config.levels-1))
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, hold3, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 125, 128])
Integer 'combine_factor': Value=4
combine_factor: 4, combine_factor // self.combine_factor: 2

==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 125, 128])
Integer 'combine_factor': Value=2
combine_factor: 2, combine_factor // self.combine_factor: 1

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 63, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 63, 256])
==========Exiting ReshapeModule.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 63, 128])
==========Exiting CombineEmbeddings.forward==========

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 32, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 32, 256])
==========Exiting ReshapeModule.forward==========

Outputs:
Tensor 'output' shape: torch.

### This is how Body.concept_matchup() will use this class once it's up and working

basically it just wants a single vector and there definitely won't be any padding to be done so it can skip most of the logic & get right to the operation chain

In [45]:
hold1 = config.combine_type
hold2 = config.combine_factor
config.combine_type = 'mlp->sum->mlp->norm'
config.combine_factor = 4
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
#module.operation_chain.mlp_0.enable_logging()
#module.operation_chain.sum_1.enable_logging()
#module.operation_chain.mlp_2.enable_logging()
#module.operation_chain.norm_3.enable_logging()

x = torch.randn(32, config.combine_factor, config.embed_dim)
output = module(x, config.combine_factor)
config.combine_type = hold1
config.combine_factor = hold2
del padding_vector, module, hold1, hold2, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 4, 128])
Integer 'combine_factor': Value=4

Outputs:
Tensor 'output' shape: torch.Size([32, 1, 128])
==========Exiting CombineEmbeddings.forward==========


In [46]:
hold1 = config.combine_type
hold2 = config.combine_factor
hold3 = config.levels
config.combine_type = 'reshape->mlp_post_reshape'
config.combine_factor = 2
config.levels = 3
padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
module = CombineEmbeddings(config, padding_vector)

module.enable_logging()
module.operation_chain.reshape_0.enable_logging()
module.operation_chain.mlp_post_reshape_1.enable_logging()

x = torch.randn(32, config.combine_factor**(config.levels-1), config.embed_dim)
output = module(x, config.combine_factor**(config.levels-1))
config.combine_type = hold1
config.combine_factor = hold2
config.levels = hold3
del padding_vector, module, hold1, hold2, hold3, x, output
module = None


==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 4, 128])
Integer 'combine_factor': Value=4

==========Entering CombineEmbeddings.forward==========
Inputs:
Tensor 'tensor' shape: torch.Size([32, 4, 128])
Integer 'combine_factor': Value=2

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 2, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 2, 256])
==========Exiting ReshapeModule.forward==========

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 2, 256])

Outputs:
Tensor 'output' shape: torch.Size([32, 2, 128])
==========Exiting MLP.forward==========

Outputs:
Tensor 'output' shape: torch.Size([32, 2, 128])
==========Exiting CombineEmbeddings.forward==========

==========Entering ReshapeModule.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 1, 2, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 1, 256])
==========Exiting ReshapeMo

# Body

In [13]:
# to prevent the warning statement from printing hella times
#cvec_warning = False

class Body(LoggingModule):
    def __init__(self, config: Config, embedding: torch.Tensor, embedding_combiner: LoggingModule):
        super().__init__()
        self.max_seq_len = config.max_seq_len
        self.combine_factor = config.combine_factor
        self.levels = config.levels
        self.embedding = embedding
        
        # Initialize a sequence of Layer instances as specified by the number of hidden layers in the config
        self.layers = nn.ModuleList(Layer(config) for _ in range(config.num_layers))

        # initialize the final normalizations to stabilize residual outputs before output layer
        self.final_norms = nn.ModuleList(Norm(config) for _ in range(config.levels))

        # initialize the concept output layers as either full MLPs or simple linear layers. maybe later make an option to make them shared?
        if config.output_layer == 'mlp':
            self.concept_output_layers = nn.ModuleList(MLP(config.embed_dim,
                                                           config.mlp_multiplier,
                                                           config.embed_dim,
                                                           config.dropout_rate) for _ in range(config.levels-1))
        else: # defaults to a linear layer
            self.concept_output_layers = nn.ModuleList(nn.Linear(config.embed_dim, config.embed_dim) for _ in range(config.levels-1))

        # gets used during inference for the matchup to concept embedding vectors
        self.concept_creator = embedding_combiner
        
    @log_io
    def forward(self,
                x0s: Tuple[torch.Tensor], # ordered from tokens -> highest concepts
                targets: Tuple[torch.Tensor] = None,
                cvec_samples: int = None,
                cvec_greedy: bool = False,
                cvec_temp: float = 1.0,
               ) -> Tuple[torch.Tensor]:
        return self.forward_training(x0s, targets) if targets is not None else self.forward_inference(x0s, cvec_samples, cvec_greedy, cvec_temp)
        
    @log_io
    def forward_training(self,
                         x0s: Tuple[torch.Tensor], # ordered from tokens -> highest concepts
                         targets: Tuple[torch.Tensor],
                        ) -> Tuple[torch.Tensor]:
        # initiate tuple to hold final residual states
        xfs = ()
        # iterate through model levels, starting from highest level concepts & ending at lowest level tokens
        for i, x in enumerate(reversed(x0s)): # reversed() makes us start at highest level

            if i == 0:
                # if we're dealing with the highest level concepts, there's nothing to cross-attend to
                x = self.layers_loop(x, i, c = None, training = True)
            else:
                # the current level x will cross-attend to the higher level c
                x = self.layers_loop(x, i, c = targets[-i], training = True)
                
            # every level gets its own norm? sure why not. only does anything if config.norm_affine == True
            x = self.final_norms[i](x, training = True)

            if i == len(x0s)-1:
                # if we're dealing with the token residual state then we use the transposed embedding matrix as our output
                x = x @ self.embedding.t()
            else:
                # all the concept levels get their own output layer to help with their regression goal
                x = self.concept_output_layers[i](x) # not setting training=True because nn.Linear doesn't know what that is
            
            # add the final residual state of the level to our tuple
            xfs += (x,)
                
        return xfs # return all final residual states
        
    @log_io
    def forward_inference(self,
                          x0s: Tuple[torch.Tensor], # ordered from tokens -> highest concepts
                          cvec_samples: int = None,
                          cvec_greedy: bool = False,
                          cvec_temp: float = 1.0,
                         ) -> Tuple[torch.Tensor]:
        # initiate tuple to hold final residual states
        xfs = ()
        # iterate through model levels, starting from highest level concepts & ending at lowest level tokens
        for i, x in enumerate(reversed(x0s)): # reversed() makes us start at highest level
            
            # if we're dealing with a concept level, then we need to fill in entire concept sequences for lower level to attend to. for token level we only want one pass
            if i != len(x0s)-1:

                # figuring out how many times we need to run this concept level in order for it to be usable for cross-attention to the layer below
                effective_max_seq_len = self.max_seq_len // (self.combine_factor ** (self.levels-1-i))
                assert x.shape[1] <= effective_max_seq_len, f'at level {i} a too-long sequence ({x.shape[1]} vs {effective_max_seq_len}) made it to Body'
                extra_runs = effective_max_seq_len - x.shape[1]

                # if extra_runs == 0 then this will only run once
                for k in range(extra_runs+1): 
                    
                    # run through layers. xfs[i-1] are the higher level concepts to pay attention to
                    x_ = self.layers_loop(x, i, training=False) if i == 0 else self.layers_loop(x, i, xfs[i-1], training=False)
                    # splice out the final prediction
                    x_ = x_[:,-1,:]
                    # normalize it
                    x_ = self.final_norms[i](x_, training=False)
                    # our concept output layer
                    x_ = self.concept_output_layers[i](x_)
                    # either select most similar concept vectors to be appended to the sequence or use the raw regression output
                    c = self.concept_matchup(x_, cvec_samples, cvec_greedy, cvec_temp) if cvec_samples is not None else x_
                    # append to x
                    x = torch.concat([x, c.unsqueeze(1)], dim=1)

                # this should only trim off one concept vector from the beginning of the sequence at most
                x = x[:,-effective_max_seq_len:,:]
                
            else: # if we're dealing with the token layer rather than a concept layer, we only want to run once
                # run the model
                x = self.layers_loop(x, i, xfs[i-1], training=False)
                
                # normalize
                x = self.final_norms[i](x, training=False)
                # the output layer is the transposed embedding matrix
                
                x = x @ self.embedding.t()
            
            # add the final residual state of the level to our tuple
            xfs += (x,)
            
        return xfs # we could return just xfs[-1] since it's inference but i have a feeling i'll want to analyze the concepts later

    @log_io
    def layers_loop(self, x: torch.Tensor,
                    i: int,
                    c: torch.Tensor = None,
                    training : bool = False,
                   ) -> torch.Tensor:
        
        # Iteratively process the input through each Layer of the model
        for layer in self.layers:
            
            # run through layers. at i==0 there's no higher level to attend to
            x = layer(x, training=training) if i == 0 else layer(x, c, training)
                
        return x

    @log_io # NOT CURRENTLY WORKING
    def concept_matchup(self,
                        c: torch.Tensor,
                        cvec_samples: int,
                        cvec_greedy: bool,
                        cvec_temp: float,
                        ) -> torch.Tensor:
        global cvec_warning
        batch_size, d = c.size()
        vocab_size = self.embedding.size(0)
    
        # Batch cosine similarity
        # Reshape c: (batch_size x 1 x embedding_dim)
        # Reshape embedding: (1 x vocab_size x embedding_dim)
        # Resulting similarity: (batch_size x vocab_size)
        token_similarities = F.cosine_similarity(c.unsqueeze(1), self.embedding.unsqueeze(0), dim=-1)
        
        # how many tokens will we sample to build up our chosen concept vector?
        if cvec_samples is None:
            cvec_samples = self.combine_factor ** (self.levels-1)
            if (cvec_warning == False) or (cvec_warning is None):
                print(f"cvec_samples not defined. defaulting to highest level's minimum size: combo**(levels-1) = {cvec_samples}")
                cvec_warning = True
        assert cvec_samples >= self.combine_factor ** (self.levels-1), f'cvec_samples = {cvec_samples} needs to be >= self.combine_factor ** (self.levels-1) = {self.combine_factor ** (self.levels-1)}'
        
        # Select top-k token embeddings for each concept vector
        topk_token_indices = torch.topk(token_similarities, k=cvec_samples, dim=1).indices  # (batch_size x sample)
        

### old shitty concept matchup code

```Python
#### OLD shit. not only is it inefficient, but also it's designed to only do sums to get concepts and not use the learnable padding vector
    @log_io
    def concept_matchup(self,
                        c: torch.Tensor,
                        cvec_samples: int,
                        cvec_greedy: bool,
                        cvec_temp: float,
                        ) -> torch.Tensor:
        global cvec_warning
        batch_size, d = c.size()
        vocab_size = self.embedding.size(0)
        embedding = self.embedding
    
        # Batch cosine similarity
        # Reshape c: (batch_size x 1 x embedding_dim)
        # Reshape embedding: (1 x vocab_size x embedding_dim)
        # Resulting similarity: (batch_size x vocab_size)
        token_similarities = F.cosine_similarity(c.unsqueeze(1), embedding.unsqueeze(0), dim=-1)
        
        # how many tokens will we sample to build up our chosen concept vector?
        if cvec_samples is None:
            cvec_samples = self.combine_factor ** (self.levels-1)
            if (cvec_warning == False) or (cvec_warning is None):
                print(f"cvec_samples not defined. defaulting to highest level's minimum size: combo**(levels-1) = {cvec_samples}")
                cvec_warning = True
        assert cvec_samples >= self.combine_factor ** (self.levels-1), f'cvec_samples = {cvec_samples} needs to be >= self.combine_factor ** (self.levels-1) = {self.combine_factor ** (self.levels-1)}'
        
        # Select top-k token embeddings for each concept vector
        topk_token_indices = torch.topk(token_similarities, k=cvec_samples, dim=1).indices  # (batch_size x sample)
    
        # Generate concept embeddings for each set of top-k token embeddings
        concept_embeddings_batch = []
        X_sizes_batch = []
        for i in range(batch_size):
            # Pass the list of indices for each concept
            concept_embeddings, X_sizes = self.create_concept_embeddings(embedding, 
                                                                         [topk_token_indices[i].tolist()])
            concept_embeddings_batch.append(concept_embeddings.squeeze(0))  # Remove the extra batch dimension
            X_sizes_batch.append(X_sizes)
    
        # Convert list of tensors to a tensor
        concept_embeddings_batch = torch.stack(concept_embeddings_batch)  # (batch_size x max_X_size x d)
    
        # Calculate concept similarities for each concept in the batch
        concept_similarities_batch = F.cosine_similarity(c.unsqueeze(1), concept_embeddings_batch, dim=-1)
    
        # Select the best matching concept embedding for each concept vector in the batch
        if cvec_greedy:
            best_concept_indices = concept_similarities_batch.argmax(dim=1)
            matched_concepts = concept_embeddings_batch[torch.arange(batch_size), best_concept_indices]
        else:
            # Apply softmax with temperature and sample
            topk_concept_probs = F.softmax(concept_similarities_batch / cvec_temp, dim=1)
            concept_topk_idx = torch.multinomial(topk_concept_probs, num_samples=1).squeeze(1)
            matched_concepts = concept_embeddings_batch[torch.arange(batch_size), concept_topk_idx]
    
        return matched_concepts

    @log_io
    def create_concept_embeddings(self, E: torch.Tensor, indices: torch.Tensor):
        """
        Create concept embeddings for a batch of indices.
    
        E: Embedding matrix (vocab_size x embedding_dim)
        indices: A list of lists of indices (batch_size x num_indices)
        """
        batch_size = len(indices)
        d = E.size(1)
        X_sizes = [(len(ind) - 1) * len(ind) // 2 for ind in indices]
        max_X_size = max(X_sizes)
        X = torch.empty((batch_size, max_X_size, d), dtype=E.dtype)
        
        # this could prolly be done way more efficiently with tensor operations
        for b in range(batch_size):
            count = 0
            for i in range(len(indices[b])):
                for j in range(i + 1, len(indices[b])):
                    X[b, count] = E[indices[b][i]] + E[indices[b][j]]
                    count += 1
            # Padding the rest if necessary
            if count < max_X_size:
                X[b, count:] = torch.zeros((max_X_size - count, d))
        
        # X_sizes is not useful rn but i think it may be later when we switch away from TinyShakespeare
        # and over to data that actually has variable sequence lengths
        return X, X_sizes
### optimized version of old shit. still prolly useless
    @log_io
    def optimized_concept_matchup(c: torch.Tensor, embedding: torch.Tensor, cvec_samples: int, levels: int, combo: int, cvec_greedy: bool, cvec_temp: float):
        batch_size, d = c.size()
        vocab_size = embedding.size(0)
        embedding = embedding

        # Batch cosine similarity
        # Reshape c: (batch_size x 1 x d)
        # Reshape embedding: (1 x vocab_size x d)
        # Resulting similarity: (batch_size x vocab_size)
        token_similarities = F.cosine_similarity(c.unsqueeze(1), embedding.unsqueeze(0), dim=-1)
    
        # how many tokens will we sample to build up our chosen concept vector?
        if cvec_samples is None:
            cvec_samples = self.combine_factor ** (self.levels-1)
            if (cvec_warning == False) or (cvec_warning is None):
                print(f"cvec_samples not defined. defaulting to highest level's minimum size: combo**(levels-1) = {cvec_samples}")
                cvec_warning = True
        assert cvec_samples >= self.combine_factor ** (self.levels-1), f'cvec_samples = {cvec_samples} needs to be >= self.combine_factor ** (self.levels-1) = {self.combine_factor ** (self.levels-1)}'
        
        # Get top-k token indices and values for each concept vector
        topk_token_indices = torch.topk(token_similarities, k=cvec_samples, dim=1).indices  # (batch_size x sample)
    
        # Create concept embeddings for the entire batch
        # Repeat embedding to match the batch size and reshape for gathering
        expanded_embeddings = embedding.repeat(batch_size, 1, 1) # batch_size x vocab_size x d)???
        gathered_embeddings = torch.gather(expanded_embeddings, 1, topk_token_indices.unsqueeze(2).expand(-1, -1, d))
    
        # Compute sum of embeddings for each pair to create concept embeddings
        # Reshape to (batch_size, cvec_samples, cvec_samples, d) for pairwise addition
        concept_embeddings = gathered_embeddings.unsqueeze(2) + gathered_embeddings.unsqueeze(1)
        concept_embeddings = concept_embeddings.reshape(batch_size, cvec_samples ** 2, d)
    
        # Calculate concept similarities
        concept_similarities = F.cosine_similarity(c.unsqueeze(1), concept_embeddings, dim=-1)
    
        # Select the best matching concept embedding
        if cvec_greedy:
            best_concept_indices = concept_similarities.argmax(dim=1)
            matched_concepts = concept_embeddings[torch.arange(batch_size), best_concept_indices]
        else:
            # Apply softmax with temperature and sample
            topk_concept_probs = F.softmax(concept_similarities / cvec_temp, dim=1)
            concept_topk_idx = torch.multinomial(topk_concept_probs, num_samples=1).squeeze(1)
            matched_concepts = concept_embeddings[torch.arange(batch_size), concept_topk_idx]
    
        return matched_concepts

    @log_io
    def optimized_create_concept_embeddings(E: torch.Tensor, indices: torch.Tensor):
        """
        Create concept embeddings for a batch of indices in a vectorized manner.
    
        E: Embedding matrix (vocab_size x embedding_dim)
        indices: A list of lists of indices (batch_size x num_indices)
        """
        batch_size = len(indices)
        max_indices = max(len(ind) for ind in indices)
        d = E.size(1)
    
        # Expand E to match the batch size and max indices
        expanded_E = E.unsqueeze(0).expand(batch_size, -1, -1)  # (batch_size, vocab_size, d)
    
        # Create an index tensor for gathering
        index_tensor = torch.zeros((batch_size, max_indices), dtype=torch.long)
        for i, ind in enumerate(indices):
            index_tensor[i, :len(ind)] = torch.tensor(ind, dtype=torch.long)
    
        # Gather embeddings according to the indices
        gathered_embeddings = torch.gather(expanded_E, 1, index_tensor.unsqueeze(2).expand(-1, -1, d))
    
        # Calculate pairwise sums of the embeddings
        pairwise_sums = gathered_embeddings.unsqueeze(2) + gathered_embeddings.unsqueeze(1)
        # Remove self-summation pairs and duplicates (upper triangular part without diagonal)
        concept_embeddings = pairwise_sums.triu(1)
    
        # Reshape and remove zero rows (due to the upper triangular operation and varying lengths)
        concept_embeddings = concept_embeddings.reshape(batch_size, max_indices * max_indices, d)
        non_zero_rows = concept_embeddings.abs().sum(dim=2) != 0
        concept_embeddings = torch.stack([concept_embeddings[i][non_zero_rows[i]] for i in range(batch_size)])
    
        return concept_embeddings
```

## demonstrations/debugging

### training

In [39]:
with torch.no_grad():
    # first let's do 2 levels training
    embedder = nn.Embedding(config.vocab_size, config.embed_dim)
    padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
    embedding_combiner = CombineEmbeddings(config, padding_vector)
    hold = config.levels
    config.levels = 2
    module = Body(config, embedder.weight, embedding_combiner)
    module.enable_logging()
    
    ### enabling logging for sub-modules
    #module.layers[0].enable_logging()
    #module.final_norms[0].enable_logging()
    
    ### disabling logging for sub-functions
    module.disable_function_logging('forward')
    #module.disable_function_logging('forward_training')
    #module.disable_function_logging('forward_inference')
    #module.disable_function_logging('layers_loop')
    #module.disable_function_logging('concept_matchup')
    #module.disable_function_logging('create_concept_embeddings')
    
    x = torch.randn(32, config.max_seq_len, config.embed_dim)
    c = torch.randn(32, config.max_seq_len // config.combine_factor, config.embed_dim)
    x0s = (x,c)
    targets = (x + torch.randn_like(x), c + torch.randn_like(c))
    output = module(x0s, targets)
    config.levels = hold
del embedder, embedding_combiner, hold, module, x, c, x0s, targets, output
embedder, embedding_combiner, module = None, None, None


==========Entering Body.forward_training==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 256, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 64, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])
Integer 'i': Value=0
Other-type 'c': Type=NoneType, Value=None
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting Body.layers_loop==========

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 256, 128])
Integer 'i': Value=1
Tensor 'c' shape: torch.Size([32, 64, 128])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 256, 128])
==========Exiting Body.layers_loop==========

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 64, 128])
Tensor 'output[1]' shape: to

In [40]:
with torch.no_grad():
    # 3 levels training
    embedder = nn.Embedding(config.vocab_size, config.embed_dim)
    padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
    embedding_combiner = CombineEmbeddings(config, padding_vector)
    hold = config.levels
    config.levels = 3
    module = Body(config, embedder.weight, embedding_combiner)
    module.enable_logging()
    
    ### enabling logging for sub-modules
    #module.layers[0].enable_logging()
    #module.final_norms[0].enable_logging()
    
    ### disabling logging for sub-functions
    module.disable_function_logging('forward')
    #module.disable_function_logging('forward_training')
    #module.disable_function_logging('forward_inference')
    #module.disable_function_logging('layers_loop')
    #module.disable_function_logging('concept_matchup')
    #module.disable_function_logging('create_concept_embeddings')
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor**0), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
    x0s = (x0, c1, c2)
    targets = (x0 + torch.randn_like(x0), c1 + torch.randn_like(c1), c2 + torch.randn_like(c2))
    output = module(x0s, targets)
    config.levels = hold
del embedder, embedding_combiner, hold, module, x0, c1, c2, x0s, targets, output
embedder, embedding_combiner, module = None, None, None


==========Entering Body.forward_training==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 256, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 64, 128])
    Tensor 'x0s[2]' shape: torch.Size([32, 16, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])
    Tensor 'targets[2]' shape: torch.Size([32, 16, 128])

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 16, 128])
Integer 'i': Value=0
Other-type 'c': Type=NoneType, Value=None
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 16, 128])
==========Exiting Body.layers_loop==========

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])
Integer 'i': Value=1
Tensor 'c' shape: torch.Size([32, 16, 128])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting Body.laye

In [41]:
with torch.no_grad():
    # 4 levels training
    embedder = nn.Embedding(config.vocab_size, config.embed_dim)
    padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
    embedding_combiner = CombineEmbeddings(config, padding_vector)
    hold = config.levels
    config.levels = 4
    module = Body(config, embedder.weight, embedding_combiner)
    module.enable_logging()
    
    ### enabling logging for sub-modules
    #module.layers[0].enable_logging()
    #module.final_norms[0].enable_logging()
    
    ### disabling logging for sub-functions
    module.disable_function_logging('forward')
    #module.disable_function_logging('forward_training')
    #module.disable_function_logging('forward_inference')
    #module.disable_function_logging('layers_loop')
    #module.disable_function_logging('concept_matchup')
    #module.disable_function_logging('create_concept_embeddings')
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor**0), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
    c3 = torch.randn(32, config.max_seq_len // (config.combine_factor**3), config.embed_dim)
    x0s = (x0, c1, c2, c3)
    targets = (x0 + torch.randn_like(x0), c1 + torch.randn_like(c1), c2 + torch.randn_like(c2), c3 + torch.randn_like(c3))
    output = module(x0s, targets)
    config.levels = hold
del embedder, embedding_combiner, hold, module, x0, c1, c2, c3, x0s, targets, output
embedder, embedding_combiner, module = None, None, None


==========Entering Body.forward_training==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 256, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 64, 128])
    Tensor 'x0s[2]' shape: torch.Size([32, 16, 128])
    Tensor 'x0s[3]' shape: torch.Size([32, 4, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])
    Tensor 'targets[2]' shape: torch.Size([32, 16, 128])
    Tensor 'targets[3]' shape: torch.Size([32, 4, 128])

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 4, 128])
Integer 'i': Value=0
Other-type 'c': Type=NoneType, Value=None
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 4, 128])
==========Exiting Body.layers_loop==========

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 16, 128])
Integer 'i': Value=1
Tensor 'c' shape: torch.Size([32, 4, 128])
Integer 'tra

### inference w/out concept matchup

In [43]:
with torch.no_grad():
    # now 2 levels partial sequence length, so like we're doing inference
    # our partial sequence will always be a clean interval of config.combine_factor because Model.create_x0s() uses CombineEmbeddings() to make the intervals clean
    embedder = nn.Embedding(config.vocab_size, config.embed_dim)
    padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
    embedding_combiner = CombineEmbeddings(config, padding_vector)
    hold = config.levels
    config.levels = 2
    module = Body(config, embedder.weight, embedding_combiner)
    module.enable_logging()
    
    ### enabling logging for sub-modules
    #module.layers[0].enable_logging()
    #module.final_norms[0].enable_logging()
    
    ### disabling logging for sub-functions
    #module.disable_function_logging('forward')
    #module.disable_function_logging('forward_training')
    #module.disable_function_logging('forward_inference')
    #module.disable_function_logging('layers_loop')
    #module.disable_function_logging('concept_matchup')
    module.disable_function_logging('create_concept_embeddings') # rn this one is hella inefficient using a for loop over batch so not fun to print
    
    x = torch.randn(32, config.max_seq_len - (config.combine_factor**2), config.embed_dim)
    c = torch.randn(32, (config.max_seq_len // config.combine_factor) - config.combine_factor, config.embed_dim)
    x0s = (x,c)
    output = module(x0s, cvec_samples = None)
    config.levels = hold
del embedder, embedding_combiner, hold, module, x, c, x0s, output
embedder, embedding_combiner, module = None, None, None



==========Entering Body.forward==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 240, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 60, 128])
Other-type 'targets': Type=NoneType, Value=None

==========Entering Body.forward_inference==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 240, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 60, 128])
Other-type 'cvec_samples': Type=NoneType, Value=None
Integer 'cvec_greedy': Value=False
Float 'cvec_temp': Value=1.0

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 60, 128])
Integer 'i': Value=0
Integer 'c': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 60, 128])
==========Exiting Body.layers_loop==========

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 61, 128])
Integer 'i': Value=0
Integer 'c': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 61, 128])
==========Exiting Body.layers_

In [44]:
with torch.no_grad():
    # now 3 levels partial sequence length, so like we're doing inference
    # our partial sequence will always be a clean interval of config.combine_factor because Model.create_x0s() uses CombineEmbeddings() to make the intervals clean
    embedder = nn.Embedding(config.vocab_size, config.embed_dim)
    padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
    embedding_combiner = CombineEmbeddings(config, padding_vector)
    hold = config.levels
    config.levels = 3
    module = Body(config, embedder.weight, embedding_combiner)
    module.enable_logging()
    
    ### enabling logging for sub-modules
    #module.layers[0].enable_logging()
    #module.final_norms[0].enable_logging()
    
    ### disabling logging for sub-functions
    #module.disable_function_logging('forward')
    #module.disable_function_logging('forward_training')
    #module.disable_function_logging('forward_inference')
    #module.disable_function_logging('layers_loop')
    module.disable_function_logging('concept_matchup') # this one is pretty boring and our prints are already long enough
    module.disable_function_logging('create_concept_embeddings') # rn this one is hella inefficient using a for loop over batch so not fun to print
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor ** 0) - (config.combine_factor**3), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor ** 1) - (config.combine_factor**2), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor ** 2) - (config.combine_factor**1), config.embed_dim)
    x0s = (x0, c1, c2)
    output = module(x0s, cvec_samples = None)
    config.levels = hold
del embedder, embedding_combiner, hold, module, x0, c1, c2, x0s, output
embedder, embedding_combiner, module = None, None, None


==========Entering Body.forward==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 192, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 48, 128])
    Tensor 'x0s[2]' shape: torch.Size([32, 12, 128])
Other-type 'targets': Type=NoneType, Value=None

==========Entering Body.forward_inference==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 192, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 48, 128])
    Tensor 'x0s[2]' shape: torch.Size([32, 12, 128])
Other-type 'cvec_samples': Type=NoneType, Value=None
Integer 'cvec_greedy': Value=False
Float 'cvec_temp': Value=1.0

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 12, 128])
Integer 'i': Value=0
Integer 'c': Value=False

Outputs:
Tensor 'output' shape: torch.Size([32, 12, 128])
==========Exiting Body.layers_loop==========

==========Entering Body.layers_loop==========
Inputs:
Tensor 'x' shape: torch.Size([32, 13, 128])
Integer 'i': Value=0
Integer 

### infernce w/ matchup

# Concept Loss

In [24]:
class ConceptLoss(LoggingModule):
    def __init__(self, config: Config):
        super().__init__()
        self.combine_factor = config.combine_factor
        self.levels = config.levels
        self.level_loss_weight = config.level_loss_weight

        self.MAE_loss = nn.L1Loss() if config.mae_loss else None
        self.MSE_loss = nn.MSELoss() if config.mse_loss else None
        self.COS_loss = nn.CosineSimilarity(dim=-1, eps=1e-6) if config.cos_loss else None
    
    @log_io
    def forward(self, 
                xfs: Tuple[torch.Tensor], # xfs are ordered highest concept level -> token level
                targets: Tuple[torch.Tensor], # targets are ordered highest concept level -> token level
               ) -> torch.Tensor:
        # initialize loss value
        concept_loss = torch.tensor(0.0)
        
        # iterate through all concept-embedding layers and calculate loss
        for i in range(self.levels - 1):
            # select our relevant final residual state and target vectors
            lvl_output = xfs[i]
            lvl_targets = targets[i].detach().clone()
            print(lvl_output.shape, lvl_targets.shape)
            
            # calculate the decay value placed on this level's total amount of loss
            lambadada = (self.level_loss_weight ** (self.levels -1 -i))
            
            # setup flattening for if we're doing MAE or MSE
            if (self.MAE_loss is not None) or (self.MSE_loss is not None):
                # Reshape output and target_vectors to combine batch and seq_len dimensions
                lvl_output_flat = lvl_output.view(-1, lvl_output.size(-1))
                lvl_targets_flat = lvl_targets.view(-1, lvl_targets.size(-1))

            # calculate loss values. notice multiple might occur or even none at all
            if self.MAE_loss is not None:
                concept_loss = concept_loss + self.MAE_loss(lvl_output_flat, lvl_targets_flat) * lambadada
            if self.MSE_loss is not None:
                concept_loss = concept_loss + self.MSE_loss(lvl_output_flat, lvl_targets_flat) * lambadada
            if self.COS_loss is not None:
                cosine_loss = (1 - self.COS_loss(lvl_output, lvl_targets)).mean()
                concept_loss = concept_loss + cosine_loss * lambadada

        return concept_loss

## Demonstration/Debugging

In [46]:
with torch.no_grad():
    hold1 = config.levels
    hold2 = config.cos_loss
    hold3 = config.mse_loss
    hold4 = config.mae_loss
    config.levels = 3
    config.cos_loss = True
    config.mse_loss = False
    config.mae_loss = False
    module = ConceptLoss(config)
    module.enable_logging()
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor**0), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
    xfs = (x0, c1, c2)
    targets = (x0 + torch.randn_like(x0), c1 + torch.randn_like(c1), c2 + torch.randn_like(c2))
    output = module(xfs, targets)
    print(output)
    config.levels = hold1
    config.cos_loss = hold2
    config.mse_loss = hold3
    config.mae_loss = hold4
del hold1, hold2, hold3, hold4, module, x0, c1, c2, xfs, targets, output
module = None


==========Entering ConceptLoss.forward==========
Inputs:
Tuple 'xfs':
    Tensor 'xfs[0]' shape: torch.Size([32, 256, 128])
    Tensor 'xfs[1]' shape: torch.Size([32, 64, 128])
    Tensor 'xfs[2]' shape: torch.Size([32, 16, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])
    Tensor 'targets[2]' shape: torch.Size([32, 16, 128])

Outputs:
Tensor 'output' shape: torch.Size([])
==========Exiting ConceptLoss.forward==========
tensor(0.5887)


In [47]:
with torch.no_grad():
    hold1 = config.levels
    hold2 = config.cos_loss
    hold3 = config.mse_loss
    hold4 = config.mae_loss
    config.levels = 3
    config.cos_loss = False
    config.mse_loss = True
    config.mae_loss = False
    module = ConceptLoss(config)
    module.enable_logging()
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor**0), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
    xfs = (x0, c1, c2)
    targets = (x0 + torch.randn_like(x0), c1 + torch.randn_like(c1), c2 + torch.randn_like(c2))
    output = module(xfs, targets)
    print(output)
    config.levels = hold1
    config.cos_loss = hold2
    config.mse_loss = hold3
    config.mae_loss = hold4
del hold1, hold2, hold3, hold4, module, x0, c1, c2, xfs, targets, output
module = None


==========Entering ConceptLoss.forward==========
Inputs:
Tuple 'xfs':
    Tensor 'xfs[0]' shape: torch.Size([32, 256, 128])
    Tensor 'xfs[1]' shape: torch.Size([32, 64, 128])
    Tensor 'xfs[2]' shape: torch.Size([32, 16, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])
    Tensor 'targets[2]' shape: torch.Size([32, 16, 128])

Outputs:
Tensor 'output' shape: torch.Size([])
==========Exiting ConceptLoss.forward==========
tensor(2.0027)


In [48]:
with torch.no_grad():
    hold1 = config.levels
    hold2 = config.cos_loss
    hold3 = config.mse_loss
    hold4 = config.mae_loss
    config.levels = 3
    config.cos_loss = False
    config.mse_loss = False
    config.mae_loss = True
    module = ConceptLoss(config)
    module.enable_logging()
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor**0), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
    xfs = (x0, c1, c2)
    targets = (x0 + torch.randn_like(x0), c1 + torch.randn_like(c1), c2 + torch.randn_like(c2))
    output = module(xfs, targets)
    print(output)
    config.levels = hold1
    config.cos_loss = hold2
    config.mse_loss = hold3
    config.mae_loss = hold4
del hold1, hold2, hold3, hold4, module, x0, c1, c2, xfs, targets, output
module = None


==========Entering ConceptLoss.forward==========
Inputs:
Tuple 'xfs':
    Tensor 'xfs[0]' shape: torch.Size([32, 256, 128])
    Tensor 'xfs[1]' shape: torch.Size([32, 64, 128])
    Tensor 'xfs[2]' shape: torch.Size([32, 16, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])
    Tensor 'targets[2]' shape: torch.Size([32, 16, 128])

Outputs:
Tensor 'output' shape: torch.Size([])
==========Exiting ConceptLoss.forward==========
tensor(1.5958)


In [49]:
with torch.no_grad():
    hold1 = config.levels
    hold2 = config.cos_loss
    hold3 = config.mse_loss
    hold4 = config.mae_loss
    config.levels = 3
    config.cos_loss = True
    config.mse_loss = True
    config.mae_loss = True
    module = ConceptLoss(config)
    module.enable_logging()
    
    x0 = torch.randn(32, config.max_seq_len // (config.combine_factor**0), config.embed_dim)
    c1 = torch.randn(32, config.max_seq_len // (config.combine_factor**1), config.embed_dim)
    c2 = torch.randn(32, config.max_seq_len // (config.combine_factor**2), config.embed_dim)
    xfs = (x0, c1, c2)
    targets = (x0 + torch.randn_like(x0), c1 + torch.randn_like(c1), c2 + torch.randn_like(c2))
    output = module(xfs, targets)
    print(output)
    config.levels = hold1
    config.cos_loss = hold2
    config.mse_loss = hold3
    config.mae_loss = hold4
del hold1, hold2, hold3, hold4, module, x0, c1, c2, xfs, targets, output
module = None


==========Entering ConceptLoss.forward==========
Inputs:
Tuple 'xfs':
    Tensor 'xfs[0]' shape: torch.Size([32, 256, 128])
    Tensor 'xfs[1]' shape: torch.Size([32, 64, 128])
    Tensor 'xfs[2]' shape: torch.Size([32, 16, 128])
Tuple 'targets':
    Tensor 'targets[0]' shape: torch.Size([32, 256, 128])
    Tensor 'targets[1]' shape: torch.Size([32, 64, 128])
    Tensor 'targets[2]' shape: torch.Size([32, 16, 128])

Outputs:
Tensor 'output' shape: torch.Size([])
==========Exiting ConceptLoss.forward==========
tensor(4.1888)


# Model

In [29]:
class Model(LoggingModule):
    def __init__(self, config: Config, tokenizer: tokenizer):
        super().__init__()
        self.config = config
        self.tokenizer = tokenizer
        
        ### hyperparameters
        self.max_seq_len = config.max_seq_len
        self.sa_head_dim = config.sa_head_dim
        self.vocab_size = config.vocab_size
        self.embed_dim = config.embed_dim
        self.combine_factor = config.combine_factor
        self.levels = config.levels
        
        ### embedding
        # the embedding matrix. for converting tokens to the first residual state, and the last residual state to logits
        self.embedder = nn.Embedding(config.vocab_size, config.embed_dim)
        # the padding vector to get used when sequence length isn't perfect
        self.padding_vector = nn.Parameter(torch.zeros(config.embed_dim), requires_grad=True)
        # the function that combines embeddings into higher level concept residual states
        self.embedding_combiner = CombineEmbeddings(config, self.padding_vector)

        ### the actual bulk of the model
        self.body = Body(config, self.embedder.weight, self.embedding_combiner)
        
        ### the loss functions
        # lowest-level token model
        self.ce_loss_fn = nn.CrossEntropyLoss()
        # concept models
        self.concept_loss_fn = ConceptLoss(config)
        
    @log_io
    def forward(self,
                input_token_ids: torch.Tensor, # a shape (batch_size, input_seq_len) list of integer token ids to run forward pass on
                target_token_ids: torch.Tensor = None, # a shape (batch_size, input_seq_len + combo ** (levels-1)) list of token ids to train on
                cvec_samples: int = None,
                cvec_greedy: bool = False,
                cvec_temp: float = 1.0,
               ) -> torch.Tensor:

        # create the tuple of initial residual states to calculate on
        x0s = self.create_x0s(input_token_ids) # x0s are ordered token level -> highest concept level
        
        if target_token_ids is None: ### if we're doing inference
            # the body of the model that iterates through the decoder & cross-attention layers
            xfs = self.body(x0s, cvec_samples=cvec_samples, cvec_greedy=cvec_greedy, cvec_temp=cvec_temp) 

            # the actual token output logits we care about
            logits = xfs[-1]
            
            # if we're not training, then we don't need to calculate loss
            loss = None
        else: ### if we're training
            assert input_token_ids.shape[1] == target_token_ids.shape[1] - (self.combine_factor ** (self.levels - 1)), f'inputs {input_token_ids.shape[1]} and targets {target_token_ids.shape[1]} have unexpected'

            # create the tuple of target embedding vectors
            targets = self.create_targets(target_token_ids, input_token_ids.shape[1]) # targets are ordered token level -> highest concept level

            # the body of the model that iterates through the decoder & cross-attention layers
            xfs = self.body(x0s, targets) # xfs are ordered highest concept level -> token level

            ### first up is regular CE token loss
            logits = xfs[-1]
            batch_size, input_len, vocab_size = logits.shape
            # splice target tokens to exclude the ones that were only to be used by concept levels
            target_token_ids_spliced = target_token_ids[:,:input_len]
            # we reshape our logits & targets before calculating cross-entropy loss
            ce_loss = self.ce_loss_fn(logits.view(batch_size*input_len, vocab_size),
                                      target_token_ids_spliced.reshape(batch_size*input_len))

            ### the new thing, a regression loss for all our concept-embedding layers
            concept_loss = self.concept_loss_fn(xfs, tuple(reversed(targets)))
            # adding it all together
            loss = ce_loss + concept_loss
        
        return logits, loss
        
    @log_io
    def create_x0s(self, input_token_ids: torch.Tensor) -> Tuple[torch.Tensor]:
        # turn the input tokens into the first residual state using the embedding matrix
        x0 = self.embedder(input_token_ids) # (batch_size, input_len, embed_dim)

        # finding the number of padding vectos we have to use at the token level to ensure the cross-attention predictive mask will line up
        remainder = x0.shape[1] % self.combine_factor
        padding_needed = 0 if remainder == 0 else self.combine_factor - remainder

        # do the actual padding for the token level
        # once i get a more complicatead tokenizer would i replace this with a <|bos|> token? Would that token be unique to each level?
        if padding_needed > 0:
            # Replicate the padding vector the necessary number of times
            padding = self.padding_vector.repeat(padding_needed, 1).unsqueeze(0).expand(x0.shape[0], -1, -1)
            x0_padded = torch.cat([padding, x0], dim=1)
        else:
            # if there's no padding to be done
            x0_padded = x0
        print('first x0_padded: ', x0_padded.shape)
        # instantiate the tuple that'll hold all the residual states
        x0s = (x0_padded * (self.embed_dim ** 0.5),) 
        
        ### iterating through levels to create each higher-level concept residual state
        for i in range(self.levels-1):
            # combine into smaller tensor by adding token (or lower level concept) embeddings together
            lvl_combo = self.combine_factor ** (i+1)
            x0c = self.embedding_combiner(x0, lvl_combo) # c stands for concept
            print('x0c: ', x0c.shape)
            # finally scale & add it to the tuple of residual states
            x0s += (x0c * (self.embed_dim ** 0.5),)
        
        return x0s

    @log_io
    def create_targets(self, target_token_ids: torch.Tensor, input_len: int) -> Tuple[torch.Tensor]:
        # turn the target tokens into the first residual state using the embedding matrix
        t0 = self.embedder(target_token_ids[:,:input_len]) # (batch_size, input_len, embed_dim)
        print('t0: ', t0.shape)
        
        # instantiate the tuple that'll hold all the residual states
        targets = (t0,) 
        
        ### iterating through levels to create each higher-level concepts
        for i in range(self.levels-1):
            # calculate the correct combo factor for this level
            lvl_combo = self.combine_factor ** (i+1)

            # my subsetting here is all messy. doesn't properly take into account off-sequences & the padding token
            # i think maybe i can fix this in the predictive mask once i make that part

            # how many tokens off are we from a perfectly sized (multiple of lvl_combo) sequence, meaning how many padding vectors do we need?
            offset = input_len % self.combine_factor # will only ever be self.combine_factor -1 at most

            # adjust input_len to ceiling the size necessary for this level
            input_len_adj = input_len + lvl_combo

            # subset the currect targets to be predicted at this level
            target_token_ids_adj = target_token_ids[:, lvl_combo - offset:lvl_combo - offset + input_len_adj]

            # turn them into embeddings
            raw_target_vectors = self.embedder(target_token_ids_adj)

            # combine the token embeddings into concepts
            c_vecs = self.embedding_combiner(raw_target_vectors, lvl_combo)
            print('c_vecs: ', c_vecs.shape)
            # append to tuple
            targets += (c_vecs,)
        
        return targets
        
    @log_io
    def generate(self,
                 prompt: str,
                 output_len: int = 1, # the model will output 1 token by default
                 temperature: float = 1.0, # 1.0 would be no effect
                 top_p: float = 1.0,
                 top_k: int = config.vocab_size,
                ) -> str: 
        """ Wrapper around sampler() that deals with manipulation of the sequence """
        # encoding the prompt into token indices
        tokens = self.tokenizer.encode(prompt)

        # turning it into the right tensor shape
        tokens = torch.tensor(tokens, device=config.device).unsqueeze(0)
        
        # we wouldn't want to go past the maximum context length we trained on
        if len(tokens) + output_len > self.config.max_seq_len:
            output_len = self.max_seq_len - len(tokens)
            print("capping output at maximum sequence length")

        for i in range(output_len):
            # get the model's output logits and ignore the loss, which would be a NoneType object
            logits, _ = self(tokens[:,:self.max_seq_len])
            
            next_token = self.Sampler(logits, temperature, top_p, top_k)

            # add our new token to the sequence
            tokens = torch.cat((tokens, next_token), dim=1)

        # resets this variable so that the corresponding warning in Body.concept_matchup can come up next time we perform inference
        global cvec_warning 
        cvec_warning = False

        # decode our list of tokens to an actual string
        return self.tokenizer.decode(tokens.squeeze(0).tolist())

    @torch.no_grad() # no need to keep track of gradients during inference
    @log_io
    def Sampler(
        self,
        logits: torch.Tensor, # shape (batch_size, input_len, vocab_size)
        temperature: float, # controls how boring vs random the outputs should be
        top_p: float, # the maximum cumulative probability of output options we're willing to consider
        top_k: int, # the maximum number of output options we're willing to consider
    ) -> torch.Tensor:
        
        # Select the last element for each sequence & apply temperature scaling
        logits = logits[:,-1,:].div_(temperature) # -> (batch_size, vocab_size)
        
        # Calculate probabilities with softmax.
        probs = torch.softmax(logits, dim=-1, dtype=torch.float) # dim=-1 is the vocab_size dimension that we calculate along
        #print('first probs: ', probs)
        
        # sort the probabilities to for use in top-p & top-k. both are (batch_size, vocab_size)
        probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)

        ### calculating top-p
        # creates same-size tensor of cumulatve probabilities instead of indivdiual probs
        probs_sum = torch.cumsum(probs_sort, dim=-1) 
        # mask where 0's are top-p selections & 1's are to be excluded
        top_ps_mask = (probs_sum - probs_sort) > top_p
        # the original probabilities with excluded tokens changed to 0.0
        probs_sort = torch.where(top_ps_mask, 0, probs_sort) 

        ### calculating top_k
        # create a shape (vocab_size) tensor that just iterates up by 1's
        top_ks_mask = torch.arange(probs_idx.shape[-1], device=probs_idx.device) 
        # expand our mask along the batch_size dimension to become size (batch_size, vocab_size)
        top_ks_mask = top_ks_mask.expand(probs_idx.shape[0], -1)
        # top_ks is a list of integers. we keep whichever entries in top_ks_mask are greater than their corresponding entries in top_ks
        top_ks_mask = top_ks_mask >= top_k

        # we'll be combining top-p with top-k and using whichever gives us fewer tokens. a very conservative approach
        # this trims probs_sort to also fit within our top_k requirement
        probs_sort = torch.where(top_ks_mask, 0, probs_sort)

        # Re-normalization so that total probabilities add up to 1
        probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
        
        # now we rearrange the modified probabilities in probs_sort back to their original order according to probs_idx
        probs = torch.gather(probs_sort, dim=-1, index=torch.argsort(probs_idx, dim=-1))
        #print('probs after topp & k: ', probs)
        
        # samples from the distribution
        next_token_id = torch.multinomial(probs, num_samples=1)
        
        return next_token_id # returns the predicted token

## demonstration/debugging

### training

In [16]:
module = Model(config, tokenizer)
module.enable_logging()

### enabling logging for sub-modules
#module.embedding_combiner.enable_logging()
#module.body.enable_logging()
#module.concept_loss_fn.enable_logging()

### disabling logging for sub-functions
#module.disable_function_logging('create_x0s')
#module.disable_function_logging('create_targets')
#module.disable_function_logging('generate')
#module.disable_function_logging('sampler')

input_token_ids = torch.randint(config.vocab_size, 
                                (32, config.max_seq_len))
target_token_ids = torch.randint(config.vocab_size, 
                                 (32, config.max_seq_len + (config.combine_factor ** (config.levels-1))))
output, loss = module(input_token_ids, target_token_ids)
print(loss)
del module, input_token_ids, target_token_ids, output, loss


==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 128])
Tensor 'target_token_ids' shape: torch.Size([32, 144])

==========Entering Model.create_x0s==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 128])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 128, 128])
Tensor 'output[1]' shape: torch.Size([32, 32, 128])
Tensor 'output[2]' shape: torch.Size([32, 8, 128])
==========Exiting Model.create_x0s==========

==========Entering Model.create_targets==========
Inputs:
Tensor 'target_token_ids' shape: torch.Size([32, 144])
Integer 'input_len': Value=128

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 128, 128])
Tensor 'output[1]' shape: torch.Size([32, 33, 128])
Tensor 'output[2]' shape: torch.Size([32, 8, 128])
==========Exiting Model.create_targets==========


/Users/tunadorable/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1056, 128])) that is different to the input size (torch.Size([1024, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (1024) must match the size of tensor b (1056) at non-singleton dimension 0

### inference

In [19]:
module = Model(config, tokenizer)
module.enable_logging()

### enabling logging for sub-modules
#module.embedding_combiner.enable_logging()
module.body.enable_logging()
#module.concept_loss_fn.enable_logging()

### disabling logging for sub-functions
#module.disable_function_logging('create_x0s')
#module.disable_function_logging('create_targets')
#module.disable_function_logging('generate')
#module.disable_function_logging('sampler')

input_token_ids = torch.randint(config.vocab_size, 
                                (32, config.max_seq_len-11))
output, loss = module(input_token_ids)
del module, input_token_ids, output, loss


==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 117])

==========Entering Model.create_x0s==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 117])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 120, 128])
Tensor 'output[1]' shape: torch.Size([32, 30, 128])
Tensor 'output[2]' shape: torch.Size([32, 8, 128])
==========Exiting Model.create_x0s==========

==========Entering Body.forward==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 120, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 30, 128])
    Tensor 'x0s[2]' shape: torch.Size([32, 8, 128])
Other-type 'targets': Type=NoneType, Value=None
Integer 'cvec_samples': Value=False
Float 'cvec_greedy': Value=1.0

==========Entering Body.forward_inference==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 120, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 30, 128])
    Tensor 'x0s[2]' shape: torch.Size([32, 8, 128])
Oth

In [72]:
module = Model(config, tokenizer)
module.enable_logging()

### enabling logging for sub-modules
#module.embedding_combiner.enable_logging()
module.body.enable_logging()
#module.concept_loss_fn.enable_logging()

### disabling logging for sub-functions
#module.disable_function_logging('create_x0s')
#module.disable_function_logging('create_targets')
#module.disable_function_logging('generate')
#module.disable_function_logging('sampler')

input_token_ids = torch.randint(config.vocab_size, 
                                (32, config.max_seq_len - 15))
output, loss = module(input_token_ids)
del module, input_token_ids, output, loss


==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 241])

==========Entering Model.create_x0s==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 241])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 244, 128])
Tensor 'output[1]' shape: torch.Size([32, 61, 128])
==========Exiting Model.create_x0s==========

==========Entering Body.forward==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 244, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 61, 128])
Other-type 'targets': Type=NoneType, Value=None
Integer 'cvec_samples': Value=False
Float 'cvec_greedy': Value=1.0

==========Entering Body.forward_inference==========
Inputs:
Tuple 'x0s':
    Tensor 'x0s[0]' shape: torch.Size([32, 244, 128])
    Tensor 'x0s[1]' shape: torch.Size([32, 61, 128])
Other-type 'cvec_samples': Type=NoneType, Value=None
Integer 'cvec_greedy': Value=False
Float 'cvec_temp': Value=1.0

==========Entering Body.layers_loop=====

# Instantiate a brand new model

In [30]:
model = Model(config, tokenizer).to(config.device)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

print(model)

2907.648 K parameters
Model(
  (embedder): Embedding(512, 128)
  (embedding_combiner): CombineEmbeddings(
    (operation_chain): Sequential(
      (reshape_0): ReshapeModule()
      (mlp_post_reshape_1): MLP(
        (gate_proj): Linear(in_features=512, out_features=1024, bias=True)
        (up_proj): Linear(in_features=512, out_features=1024, bias=True)
        (down_proj): Linear(in_features=1024, out_features=128, bias=True)
      )
      (norm_2): Norm()
    )
  )
  (body): Body(
    (layers): ModuleList(
      (0-7): 8 x Layer(
        (pre_self_mqa_norm): Norm()
        (self_mqa): selfMQA()
        (post_self_mqa_norm): Norm()
        (pre_cross_mqa_x_norm): Norm()
        (pre_cross_mqa_c_norm): Norm()
        (cross_mqa): crossMQA()
        (post_cross_mqa_norm): Norm()
        (pre_mlp_norm): Norm()
        (mlp): MLP(
          (gate_proj): Linear(in_features=128, out_features=256, bias=True)
          (up_proj): Linear(in_features=128, out_features=256, bias=True)
         

# Training

In [17]:
# Train and test splits
data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be our training dataset, the rest for validation
train_data = data[:n]
val_data = data[n:]

In [18]:
# data loading for training which generates a small batch of data of inputs x and targets y
def get_batch(split, batch_size):
    # whether we grab from our training or validation dataset
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - (config.max_seq_len + (config.combine_factor ** (config.levels-1))), (batch_size,))
    # some training batches need to be offset so it learns how to use the padding vector
    offset = 0#random.randint(0, config.combine_factor)
    x = torch.stack([data[i:i+config.max_seq_len - offset] for i in ix])
    ### i actually need the y tensor to be + (config.combine_factor ** (config.levels-1)) to fit the future concepts
    y = torch.stack([data[i+1:i+1+(config.max_seq_len + (config.combine_factor ** (config.levels-1))) - offset] for i in ix])
    x, y = x.to(config.device), y.to(config.device)
    return x, y

In [19]:
@torch.no_grad()
def estimate_loss(model, batch_size, eval_iters = 5): # to estimate loss during the training loop
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [31]:
# create a PyTorch optimizer
# this is not what they used, but this learning rate & weight decay work for our tiny minGemma
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

# how long we want to train for
config.max_iters = 10

# Learning rate scheduler setup
lr_start = config.learning_rate
lr_end = config.learning_rate_end  # Final learning rate after decay
lr_lambda = lambda iter: (lr_end + (lr_start - lr_end) * (1 - iter / config.max_iters))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# how often we want to check & see how our loss is doing
eval_interval = 2

# batch size to use
config.batch_size = 32

# ---- BOOKMARK -----
so it looks like the wrong number of target concept vectors are getting created during training. once i've fixed that i'd like to confirm training & inference are working and then switch to the new dataset

In [33]:
start_time = time.time()

# Enable anomaly detection. uncomment these lines if you need to do extensive debugging
#torch.autograd.set_detect_anomaly(True)

for iter in range(config.max_iters):

    # sample a batch of data
    xb, yb = get_batch('train', config.batch_size)
    print(xb.shape, yb.shape)
    
    # train
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

    # Update the learning rate
    scheduler.step()
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == config.max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time
        losses = estimate_loss(model, config.batch_size)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

# Disable anomaly detection after the training loop
#torch.autograd.set_detect_anomaly(False)

torch.Size([32, 128]) torch.Size([32, 144])
first x0_padded:  torch.Size([32, 128, 128])
x0c:  torch.Size([32, 32, 128])
x0c:  torch.Size([32, 8, 128])
t0:  torch.Size([32, 128, 128])
c_vecs:  torch.Size([32, 33, 128])
c_vecs:  torch.Size([32, 8, 128])
torch.Size([32, 8, 128]) torch.Size([32, 8, 128])
torch.Size([32, 32, 128]) torch.Size([32, 33, 128])


RuntimeError: The size of tensor a (1024) must match the size of tensor b (1056) at non-singleton dimension 0

# Saving your model

In [33]:
name = f'models/{model.__class__.__name__}_{time.strftime("%Y-%m-%d|%H-%M-%S")}'
torch.save(model.state_dict(), f'{name}.pth')

# Convert the dataclass object to a dictionary
config_dict = asdict(config)

# Serialize the dictionary to a JSON file
with open(f'{name}.json', 'w') as f:
    json.dump(config_dict, f)

# Load a Pretrained Model

In [ ]:
name = '?'

# Deserialize the JSON file back to a dictionary
with open(f'models/{name}.json', 'r') as f:
    config_dict = json.load(f)

# Convert the dictionary back to a dataclass object
config = Config(**config_dict)

# Initialize a blank model
model = Model(config, tokenizer).to(config.device)  

# here's the path to a minGemma model that i've trained with roughly 1m parameters
path = f'models/{name}.pth'

# Load the saved state dictionary
model.load_state_dict(torch.load(path)) 
# REMEMBER TO CHANGE VALUES IN CONFIG TO MATCH THE MODEL YOU'VE LOADED

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters')

# If you only plan to do inference, switch to evaluation mode
model.eval()

# If you plan to continue training the model, switch to training mode
#model.train()

# Inference

In [32]:
input_str = "JULIET:\nO Romeo, Romeo! wherefore art thou Romeo?\n" # the classic line
max_useable_output_len = config.max_seq_len - len(input_str)
output = model.generate(input_str, output_len = max_useable_output_len, temperature=0.9)
print(output)

JULIET:
O Romeo, Romeo! wherefore art thou Romeo?
And not full it my and the friend with for my will.
O'er a be any thy Henry after, and parge?
And no looks of ma
